<a href="https://colab.research.google.com/github/DDDS18-GTFS/ddds.18.capstone/blob/dev.Andrew/GTFS_ABQ_load_pickle_set_params_merge_fix_set_anomalies_v9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ABC note 250726-1715: I have some concerns about the anomaly placement, but I also haven't worked with a multi-day dataset previously, so I need to do some more testing.


#Load the Libraries

In [ ]:
#Required Libraries
import pandas as pd
import numpy as np

# # Install if needed
# !pip install -q ipywidgets

import ipywidgets as widgets
from IPython.display import display, clear_output

from datetime import datetime
import zipfile

from shapely.geometry import Point, LineString
from geopy.distance import geodesic

from folium import Map, FeatureGroup, CircleMarker, PolyLine, Marker, Icon, LayerControl
from matplotlib import colors as mcolors
import matplotlib.pyplot as plt
import folium

In [ ]:
# Select Parameters for Anomaly Detection
# This section sets threshholds for classifying anomalous transit events.

# --- Use meters for user input ---
# The jump_thresh_m_widget creates a sliding bar for selecting how much distance there must be between
# two consecutive positions for a single vehicle to have gone 'unusually far' or 'jumped'.
jump_thresh_m_widget = widgets.IntSlider(
    value=500,
    min=50,
    max=2000,
    step=50,
    description='Jump Distance',
    layout=widgets.Layout(width='60%')
)

# Other anomaly widgets
# The disappear_thresh_widget creates a sliding bar for selecting how many consecutive seconds  must pass
# without a vehicle transmitting data before we consider that vehicle to have been gone 'unusually long'.
disappear_thresh_widget = widgets.IntSlider(value=300, min=60, max=1800, step=30, description='Time Gap')

# The min_jump_widget creates a sliding bar for selecting how many 'jumps' a vehicle must make before
# we consider the jumping to be 'unusual'.
# SUGGESTION: can we make this based off of jump points as a percentage of total recorded points a required number of consecutive jumps?
# As we use more data, any set value of total jumps will become more and more easily met.
min_jump_widget = widgets.IntSlider(value=2, min=1, max=10, step=1, description='Min Jumps')

# The speed_thresh_widget creates a sliding bar for selecting how fast a vihicke must be going for us
# to consider that vehicle as 'unusually fast'.
speed_thresh_widget = widgets.IntSlider(value=70, min=10, max=100, step=5, description='Speed')

# motion to depricate this widget
reversal_thresh_widget = widgets.IntSlider(value=120, min=60, max=180, step=5, description='Heading Δ°')

# --- Stuck vehicle detection ---
# The stuck_speed_widget creates a sliding bar for selecting how slow a vehicle must be going
# for us to consider it as effectively unmoving.
stuck_speed_widget = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=5.0,
    step=0.1,
    description='Stuck Speed',
    layout=widgets.Layout(width='60%')
)

# The stuck_window_widget creates a sliding bar for selecting how many consecutive Stuck Speed
# events a vehicle must have before it is considered to have been still for an unusually long period.
stuck_window_widget = widgets.IntSlider(
    value=4,
    min=1,
    max=20,
    step=1,
    description='Window Size',
    layout=widgets.Layout(width='60%')
)

# --- Repeated point tolerance ---
# motion to depricate this widget
repeat_tolerance_widget = widgets.FloatLogSlider(
    value=1e-5,
    base=10,
    min=-7,  # 1e-7
    max=-3,  # 1e-3
    step=0.1,
    description='Repeat Tolerance',
    layout=widgets.Layout(width='60%')
)

# --- Early appearance margin ---
# motion to depricate this widget
early_margin_widget = widgets.IntSlider(
    value=30,
    min=0,
    max=600,
    step=10,
    description='Early Margin',
    layout=widgets.Layout(width='60%')
)

# --- Off-route buffer ---
# The offroute_buffer_widget creates a sliding bar for selecting how many meters
# a vehicle must be off of its expected route before we actually label it as off route
offroute_buffer_widget = widgets.IntSlider(
    value=50,
    min=10,
    max=500,
    step=10,
    description='Off-Route Buffer',
    layout=widgets.Layout(width='60%')
)

# Labels
jump_label = widgets.Label(value="(meters)")
disappear_label = widgets.Label(value="(seconds)")
min_jump_label = widgets.Label(value="(count)")
speed_label = widgets.Label(value="(mph)")
reversal_label = widgets.Label(value="(degrees of reversal)")
stuck_speed_label = widgets.Label(value="(mph)")
stuck_window_label = widgets.Label(value="(frames in rolling window)")
repeat_tolerance_label = widgets.Label(value="(decimal degrees)")
early_margin_label = widgets.Label(value="(seconds)")
offroute_buffer_label = widgets.Label(value="(meters)")

# Assemble UI layout
slider_widgets = widgets.VBox([
    widgets.HBox([jump_thresh_m_widget, jump_label]),
    widgets.HBox([disappear_thresh_widget, disappear_label]),
    widgets.HBox([min_jump_widget, min_jump_label]),
    widgets.HBox([speed_thresh_widget, speed_label]),
    widgets.HBox([reversal_thresh_widget, reversal_label]),
    widgets.HBox([stuck_speed_widget, stuck_speed_label]),
    widgets.HBox([stuck_window_widget, stuck_window_label]),
    widgets.HBox([repeat_tolerance_widget, repeat_tolerance_label]),
    widgets.HBox([early_margin_widget, early_margin_label]),
    widgets.HBox([offroute_buffer_widget, offroute_buffer_label]),
])

# Button and save logic
submit_button = widgets.Button(description="Save Parameters", button_style='primary')
anomaly_params = {}

def save_params(b):
    '''
    save_params displays widgets that allow you to set up and save anomaly detection threshholds.
    '''
    clear_output(wait=True)
    display(slider_widgets, submit_button)

    global anomaly_params
    jump_m = jump_thresh_m_widget.value
    jump_deg = jump_m / 111000  # Convert meters → degrees

    anomaly_params = {
        "JUMP_DISTANCE_THRESHOLD": jump_deg,
        "JUMP_DISTANCE_METERS": jump_m,
        "DISAPPEARANCE_TIME_THRESHOLD": disappear_thresh_widget.value,
        "MIN_JUMP_COUNT_PER_VEHICLE": min_jump_widget.value,
        "SPEED_LIMIT_MPH": speed_thresh_widget.value,
        "REVERSAL_HEADING_THRESHOLD": reversal_thresh_widget.value,
        "STUCK_SPEED_MPH": stuck_speed_widget.value,
        "STUCK_WINDOW_SIZE": stuck_window_widget.value,
        "REPEATED_COORD_TOLERANCE": repeat_tolerance_widget.value,
        "EARLY_APPEARANCE_MARGIN_SEC": early_margin_widget.value,
        "OFF_ROUTE_BUFFER_M": offroute_buffer_widget.value,
    }

    print("✅ Anomaly detection parameters set:")
    for k, v in anomaly_params.items():
        print(f"  {k}: {v}")

submit_button.on_click(save_params)

# Display interface
display(slider_widgets, submit_button)


Button(button_style='primary', description='Save Parameters', style=ButtonStyle())

✅ Anomaly detection parameters set:
  JUMP_DISTANCE_THRESHOLD: 0.0045045045045045045
  JUMP_DISTANCE_METERS: 500
  DISAPPEARANCE_TIME_THRESHOLD: 300
  MIN_JUMP_COUNT_PER_VEHICLE: 2
  SPEED_LIMIT_MPH: 70
  REVERSAL_HEADING_THRESHOLD: 120
  STUCK_SPEED_MPH: 1.0
  STUCK_WINDOW_SIZE: 4
  REPEATED_COORD_TOLERANCE: 1e-05
  EARLY_APPEARANCE_MARGIN_SEC: 30
  OFF_ROUTE_BUFFER_M: 50


#LOAD PICKLE

In [ ]:
import pickle

# Load a pickle file of a dataframe which contains our historic transit data.
with open("captures.p", "rb") as f:
    captured_data = pickle.load(f)


##Data Validation

In [ ]:
cd_df = captured_data
cd_df.size

13158480

In [ ]:
# cd_df.head(20)

,snapshot_id,timestamp_collected,vehicle_id,latitude,longitude,heading,speed_mph,route_short_name,trip_id,next_stop_id,next_stop_name,next_stop_sched_time
0,1,2025-07-22 11:17:02.268350,610,35.147320,-106.639650,186.0,9.65604,10,0,0,4th @ Guadalupe,11:32:57
1,1,2025-07-22 11:17:02.268350,358,35.106341,-106.693167,96.0,64.00000,11,0,0,Spanish Bit @ Montgomery (P-n-R),05:36:00
2,1,2025-07-22 11:17:02.268350,359,35.152506,-106.694041,92.0,0.00000,157,0,0,NW Transit Center (Bay C),05:31:00
3,1,2025-07-22 11:17:02.268350,383,35.058513,-106.569910,17.0,0.00000,16,601256,2801,Gibson @ San Pedro,15:18:45
4,1,2025-07-22 11:17:02.268350,627,35.106680,-106.550930,178.0,0.00000,31,0,0,Wyoming @ Northeastern,13:03:07
5,1,2025-07-22 11:17:02.268350,617,35.105550,-106.667950,76.0,64.00000,5,0,0,Spanish Bit @ Montgomery (P-n-R),05:36:00
6,1,2025-07-22 11:17:02.268350,961,35.089586,-106.732609,294.0,0.00000,54,0,0,A.T.C. (Bay L),06:21:00
7,1,2025-07-22 11:17:02.268350,368,35.105217,-106.641143,96.0,64.00000,66,0,0,Wenonah @ Tramway,05:29:00
8,1,2025-07-22 11:17:02.268350,1914,35.089850,-106.732920,88.0,0.00000,766,0,0,CUTC Bay B,05:21:00
9,1,2025-07-22 11:17:02.268350,1921,35.078768,-106.521377,115.0,56.00000,766,0,0,Wenonah @ Tramway (ART),05:18:00


In [ ]:
# # Display full list of column names
# print("Number of columns:", len(cd_df.columns))
# print("Column names:\n", cd_df.columns.tolist())

In [ ]:
# cd_df.head()

In [ ]:
# Convert timestamp to datetime
cd_df['timestamp_collected'] = pd.to_datetime(cd_df['timestamp_collected'], errors='coerce')

# 1. Shape and Column Overview
print("Shape:", cd_df.shape)
print("Columns:", cd_df.columns.tolist())

# 2. Null/Missing Value Counts
print("\nMissing Values:\n", cd_df.isnull().sum())

# 3. Duplicate Detection
duplicates = cd_df.duplicated().sum()
print("\nDuplicate rows:", duplicates)

# 4. Timestamp Range
print("\nTimestamp Range:")
print("Min:", cd_df['timestamp_collected'].min())
print("Max:", cd_df['timestamp_collected'].max())
print("Duration:", cd_df['timestamp_collected'].max() - cd_df['timestamp_collected'].min())

# 5. Latitude/Longitude Range
print("\nLatitude Range:", cd_df['latitude'].min(), "to", cd_df['latitude'].max())
print("Longitude Range:", cd_df['longitude'].min(), "to", cd_df['longitude'].max())

# 6. Speed Summary
print("\nSpeed Summary:\n", cd_df['speed_mph'].describe())


Shape: (1096540, 12)
Columns: ['snapshot_id', 'timestamp_collected', 'vehicle_id', 'latitude', 'longitude', 'heading', 'speed_mph', 'route_short_name', 'trip_id', 'next_stop_id', 'next_stop_name', 'next_stop_sched_time']

Missing Values:
 snapshot_id             0
timestamp_collected     0
vehicle_id              0
latitude                0
longitude               0
heading                 0
speed_mph               0
route_short_name        0
trip_id                 0
next_stop_id            0
next_stop_name          0
next_stop_sched_time    0
dtype: int64

Duplicate rows: 0

Timestamp Range:
Min: 2025-07-22 11:17:02.268350
Max: 2025-07-24 01:13:30.399545
Duration: 1 days 13:56:28.131195

Latitude Range: 0.0 to 35.4133163
Longitude Range: -106.79815 to 0.0

Speed Summary:
 count    1.096540e+06
mean     4.230205e+00
std      1.108069e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      8.000000e+01
Name: speed_mph, dtype: float64


In [ ]:
# # Plot the histogram and get the counts and bin edges
# counts, bins, patches = plt.hist(cd_df['longitude'], bins=2, rwidth=0.8)

# # Add labels and title
# plt.xlabel("Value")
# plt.ylabel("Count")
# plt.title("Histogram of Values with Counts per Bin")

# # Display the plot
# plt.show()

# # You can also access the counts and bin edges directly:
# print("Bin counts:", counts)
# print("Bin edges:", bins)

In [ ]:
# # Plot the histogram and get the counts and bin edges
# counts, bins, patches = plt.hist(cd_df['latitude'], bins=2, rwidth=0.8)

# # Add labels and title
# plt.xlabel("Value")
# plt.ylabel("Count")
# plt.title("Histogram of Values with Counts per Bin")

# # Display the plot
# plt.show()

# # You can also access the counts and bin edges directly:
# print("Bin counts:", counts)
# print("Bin edges:", bins)

In [ ]:
# Select rows where 'longitude' and 'latitude' have the same value
rows_with_same_values = cd_df[cd_df['longitude'] == cd_df['latitude']]
rows_without_same_values = cd_df[cd_df['longitude'] != cd_df['latitude']]

# Print the resulting DataFrame
print(rows_with_same_values)

       snapshot_id        timestamp_collected  vehicle_id  latitude  \
298              1 2025-07-22 11:17:02.268350        4001       0.0   
637              2 2025-07-22 11:17:32.471202        4001       0.0   
976              3 2025-07-22 11:18:02.682016        4001       0.0   
1315             4 2025-07-22 11:18:32.889178        4001       0.0   
1654             5 2025-07-22 11:19:03.095616        4001       0.0   
...            ...                        ...         ...       ...   
39407          116 2025-07-24 01:11:29.333676        4001       0.0   
39747          117 2025-07-24 01:11:59.591393        4001       0.0   
40087          118 2025-07-24 01:12:29.850812        4001       0.0   
40427          119 2025-07-24 01:13:00.146041        4001       0.0   
40766          120 2025-07-24 01:13:30.399545        4001       0.0   

       longitude  heading  speed_mph route_short_name trip_id next_stop_id  \
298          0.0    269.0        0.0         Off Duty       0        

In [ ]:
# rows_without_same_values.shape

In [ ]:
# cd_df.shape

####*.v8.2 update
We're removing the off-duty rows where Lat = Long = 0

In [ ]:
cd_df_copy = cd_df.copy()
cd_df_copy.shape

(1096540, 12)

In [ ]:
cd_df = cd_df[cd_df['longitude'] != cd_df['latitude']]
cd_df.shape

(1093311, 12)

##✅ Overall Status
Your dataset appears valid and cleanly structured for analysis. You can safely proceed with anomaly detection, with only minor issues to address or monitor.

| Category            | Result                               | Notes                                                                          |
| ------------------- | ------------------------------------ | ------------------------------------------------------------------------------ |
| **Shape**           | (1,096,540 rows, 12 columns)         | Substantial volume, well-sized for analysis                                    |
| **Missing Values**  | 0 missing in any column              | ✅ Excellent data integrity                                                     |
| **Duplicates**      | 0 duplicate rows                     | ✅ Clean snapshot logic                                                         |
| **Timestamp Range** | \~1.6 days                           | ⚠️ Slightly under 2 days; may be due to gaps or partial logging                |
| **Latitude Range**  | `0.0` to `35.41`                     | ⚠️ `0.0` indicates corrupted or failed GPS points — needs filtering            |
| **Longitude Range** | `-106.79` to `0.0`                   | ⚠️ Same issue — longitudes of `0.0` are invalid (off the west coast of Africa) |
| **Speed (mph)**     | Mean: `4.2`, Max: `80`, 75% at `0.0` | ⚠️ Excessive zero speeds → vehicles idle or unreported? Worth mapping          |
| **Outliers**        | Max speed = `80 mph`                 | 🔍 Borderline — worth flagging for manual inspection                           |


In [ ]:
# #Some preliminary prep for analysis

# # Remove invalid GPS coordinates
# cd_df_valid = cd_df[(cd_df['latitude'] > 30) & (cd_df['longitude'] < -100)]

# # Optional: filter out idle snapshots (speed = 0) if focusing on movement
# cd_df_moving = cd_df_valid[cd_df_valid['speed_mph'] > 0]


#Convert code for snapshot to use Pickle

In [ ]:
def filter_cd_df(df, start_time=None, end_time=None, routes=None, vehicles=None, bounds=None):
    '''
    Filters a dataframe(df) such that:
    - the "timestamp_collected" column's entries are greater than or equal to the start_time
    - the "timestamp_collected" column's entries are less than or equal to the end_time
    - the "route_short_name" column's entries are in routes
    - the "vehicle_id" column's entries are in vehicles

    Returns the filtered dataframe.
    '''
    df_filtered = df.copy()

    if start_time:
        df_filtered = df_filtered[df_filtered['timestamp_collected'] >= pd.to_datetime(start_time)]
    if end_time:
        df_filtered = df_filtered[df_filtered['timestamp_collected'] <= pd.to_datetime(end_time)]
    if routes:
        df_filtered = df_filtered[df_filtered['route_short_name'].isin(routes)]
    if vehicles:
        df_filtered = df_filtered[df_filtered['vehicle_id'].isin(vehicles)]
    if bounds:
        lat_min, lat_max, lon_min, lon_max = bounds
        df_filtered = df_filtered[
            (df_filtered['latitude'] >= lat_min) & (df_filtered['latitude'] <= lat_max) &
            (df_filtered['longitude'] >= lon_min) & (df_filtered['longitude'] <= lon_max)
        ]

    return df_filtered


In [ ]:


# --- Widgets ---
# route_widget lets you select which routes you'd like to like to plot in folium from a menu.
route_widget = widgets.SelectMultiple(
    options=sorted(cd_df['route_short_name'].unique()),
    description='Routes',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)
# vehicle_widget lets you select which vehicles' data you'd like to plot in folium from a menu.
vehicle_widget = widgets.SelectMultiple(
    options=sorted(cd_df['vehicle_id'].unique()),
    description='Vehicles',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)
# start_widget lets you select the earliest date you'd like to use data from.
start_widget = widgets.DatePicker(
    description='Start Date',
    value=cd_df['timestamp_collected'].min().date()
)
# end_widget lets you select the latest date you'd like to use data from.
end_widget = widgets.DatePicker(
    description='End Date',
    value=cd_df['timestamp_collected'].max().date()
)

# hour_slider creates a set of sliding bars for selecting the earliest and latest
# point of the day that you'd like to use data from.
hour_slider = widgets.IntRangeSlider(
    value=[0, 23],
    min=0,
    max=23,
    step=1,
    description='Hour Range',
    layout=widgets.Layout(width='60%')
)

# Makes a button, that will later be used to apply filters.
filter_button = widgets.Button(description='Apply Filters', button_style='primary')

# --- Global to store result ---
cd_df_filtered = pd.DataFrame()

# --- Filtering callback ---
def apply_filters(b):
    '''
    Uses previously defined widgets to filter a dataframe of historical transit data
    by rout_short_name, vehicle_id, and timestamp collected.
    Prints the number of filtered rows and displays the first 10 rows of the filtered dataframe.
    '''
    global cd_df_filtered
    clear_output(wait=True)
    display(route_widget, vehicle_widget, start_widget, end_widget, hour_slider, filter_button)

    df = cd_df.copy()

    # Apply filters
    if route_widget.value:
        df = df[df['route_short_name'].isin(route_widget.value)]
    if vehicle_widget.value:
        df = df[df['vehicle_id'].isin(vehicle_widget.value)]

    start_dt = pd.to_datetime(start_widget.value)
    end_dt = pd.to_datetime(end_widget.value) + pd.Timedelta(days=1)
    df = df[(df['timestamp_collected'] >= start_dt) &
            (df['timestamp_collected'] < end_dt)]

    hr_start, hr_end = hour_slider.value
    df = df[(df['timestamp_collected'].dt.hour >= hr_start) &
            (df['timestamp_collected'].dt.hour <= hr_end)]

    # Store result globally
    cd_df_filtered = df

    print(f"✅ Filtered {len(df)} rows.")
    display(df.head(10))

# Displays a filter button that calls the 'apply_filters' function
filter_button.on_click(apply_filters)

# --- Display UI ---
display(route_widget, vehicle_widget, start_widget, end_widget, hour_slider, filter_button)


SelectMultiple(description='Routes', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, …

SelectMultiple(description='Vehicles', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18…

DatePicker(value=datetime.date(2025, 7, 22), description='Start Date')

DatePicker(value=datetime.date(2025, 7, 24), description='End Date')

IntRangeSlider(value=(0, 23), description='Hour Range', layout=Layout(width='60%'), max=23)

Button(button_style='primary', description='Apply Filters', style=ButtonStyle())

✅ Filtered 1093311 rows.


,snapshot_id,timestamp_collected,vehicle_id,latitude,longitude,heading,speed_mph,route_short_name,trip_id,next_stop_id,next_stop_name,next_stop_sched_time
0,1,2025-07-22 11:17:02.268350,610,35.147320,-106.639650,186.0,9.65604,10,0,0,4th @ Guadalupe,11:32:57
1,1,2025-07-22 11:17:02.268350,358,35.106341,-106.693167,96.0,64.00000,11,0,0,Spanish Bit @ Montgomery (P-n-R),05:36:00
2,1,2025-07-22 11:17:02.268350,359,35.152506,-106.694041,92.0,0.00000,157,0,0,NW Transit Center (Bay C),05:31:00
3,1,2025-07-22 11:17:02.268350,383,35.058513,-106.569910,17.0,0.00000,16,601256,2801,Gibson @ San Pedro,15:18:45
4,1,2025-07-22 11:17:02.268350,627,35.106680,-106.550930,178.0,0.00000,31,0,0,Wyoming @ Northeastern,13:03:07
5,1,2025-07-22 11:17:02.268350,617,35.105550,-106.667950,76.0,64.00000,5,0,0,Spanish Bit @ Montgomery (P-n-R),05:36:00
6,1,2025-07-22 11:17:02.268350,961,35.089586,-106.732609,294.0,0.00000,54,0,0,A.T.C. (Bay L),06:21:00
7,1,2025-07-22 11:17:02.268350,368,35.105217,-106.641143,96.0,64.00000,66,0,0,Wenonah @ Tramway,05:29:00
8,1,2025-07-22 11:17:02.268350,1914,35.089850,-106.732920,88.0,0.00000,766,0,0,CUTC Bay B,05:21:00
9,1,2025-07-22 11:17:02.268350,1921,35.078768,-106.521377,115.0,56.00000,766,0,0,Wenonah @ Tramway (ART),05:18:00


In [ ]:
# #Sanity check following filtering
# cd_df_filtered.describe()
# # or
# cd_df_filtered['vehicle_id'].value_counts()


In [ ]:
#Pickle replacement
df_new = cd_df_filtered

#Old code:
#✅ Step 1a: Load and Inspect the New Snapshot
# new_snapshot_path = "/content/cabq_gtfs_snapshots_20250722_1415.csv"
# df_new = pd.read_csv(new_snapshot_path)
# df_new.info()
# df_new.head(3)

# #Also print the columns:
# print(df_new.columns.tolist())


In [ ]:
#✅ Step 2a: Trip ID Validity
# Counts and displays the total rows and number of rows with invalid trip ids.
# Calculates and displays the percentage of rows with valid trip ids.
df_new["trip_id"] = df_new["trip_id"].astype(str)
invalid_trip_ids = df_new["trip_id"].isin(["0", "Undetermined", "nan", "", "None"]).sum()
total_rows = len(df_new)

print(f"Total rows: {total_rows}")
print(f"Invalid trip_ids: {invalid_trip_ids}")
print(f"Percent valid trip_ids: {100 * (total_rows - invalid_trip_ids) / total_rows:.2f}%")


Total rows: 1093311
Invalid trip_ids: 921956
Percent valid trip_ids: 15.67%


In [ ]:
#*.v8.2 new calc for finding mostly Off Duty
invalid_route_ids = df_new["route_short_name"].isin(["0", "Undetermined", "nan", "", "None", "Off Duty"]).sum()
print(f"Invalid route_ids: {invalid_route_ids}")
print(f"Percent valid route_ids: {100 * (total_rows - invalid_route_ids) / total_rows:.2f}%")

Invalid route_ids: 905238
Percent valid route_ids: 17.20%


In [ ]:
invalid_ids = ["0", "Undetermined", "nan", "", "None", "Off Duty"]
df_clean_test = df_new[~df_new["trip_id"].isin(invalid_ids) & ~df_new["route_short_name"].isin(invalid_ids)].copy()
df_clean_test.shape

(171355, 12)

In [ ]:
#✅ Step 2b: Filter invalid trip_ids
invalid_trip_ids = ["0", "Undetermined", "nan", "", "None"]
df_clean_trips = df_new[~df_new["trip_id"].isin(invalid_trip_ids)].copy()
df_clean_trips.shape

(171355, 12)

In [ ]:
#So after all that we can see that all of the rows dropped by df_clean_trips cover those dropped by df_clean_test (both invalid trips and routes)
invalid_route_ids = df_clean_trips["route_short_name"].isin(["0", "Undetermined", "nan", "", "None", "Off Duty"]).sum()
print(f"Invalid route_ids: {invalid_route_ids}")
print(f"Percent valid route_ids: {100 * (total_rows - invalid_route_ids) / total_rows:.2f}%")

Invalid route_ids: 0
Percent valid route_ids: 100.00%


In [ ]:
df_clean = df_clean_test

In [ ]:
# df_clean.head()

,snapshot_id,timestamp_collected,vehicle_id,latitude,longitude,heading,speed_mph,route_short_name,trip_id,next_stop_id,next_stop_name,next_stop_sched_time
3,1,2025-07-22 11:17:02.268350,383,35.058513,-106.56991,17.0,0.0,16,601256,2801,Gibson @ San Pedro,15:18:45
342,2,2025-07-22 11:17:32.471202,383,35.058513,-106.56991,17.0,0.0,16,601256,2801,Gibson @ San Pedro,15:18:45
681,3,2025-07-22 11:18:02.682016,383,35.058513,-106.56991,17.0,0.0,16,601256,2801,Gibson @ San Pedro,15:18:45
1020,4,2025-07-22 11:18:32.889178,383,35.058513,-106.56991,17.0,0.0,16,601256,2801,Gibson @ San Pedro,15:18:45
1359,5,2025-07-22 11:19:03.095616,383,35.058513,-106.56991,17.0,0.0,16,601256,2801,Gibson @ San Pedro,15:18:45


####Cleanup note
We can skip several of the above cells, as dropping the invalid trip_ids also drops the invalid route_short_names

#Load the Static data

In [ ]:
#✅ Step 1b: Reload Static GTFS and Normalize It (ensure data types are consistent)

# Adjust if needed — make sure this is the static feed aligned with 2025-07-22
gtfs_zip_path = "/content/google_transit.zip"

with zipfile.ZipFile(gtfs_zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/gtfs_static")

trips = pd.read_csv("/content/gtfs_static/trips.txt", dtype=str)
routes = pd.read_csv("/content/gtfs_static/routes.txt", dtype=str)


In [ ]:
# trips.shape

(2139, 10)

In [ ]:
# trips.head()

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,4300,175.0.1,600305,Central & Tramway,NaN,0,53457,17128,1,1
1,4300,175.0.1,600306,Central & Tramway,NaN,0,53458,17128,1,1
2,4300,175.0.1,600307,Central & Tramway,NaN,0,53457,17128,1,1
3,4300,175.0.1,600308,Central & Tramway,NaN,0,53458,17128,1,1
4,4300,175.0.1,600309,Central & Tramway,NaN,0,53457,17128,1,1


In [ ]:
# trips.describe()

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
count,2139,2139,2139,2139,0,2139,2139,2139,2139,2139
unique,23,4,2139,35,0,2,157,67,1,1
top,4330,175.0.1,603119,Central & Unser Transit Center,NaN,0,53550,17212,1,1
freq,215,930,1,386,NaN,1075,38,108,2139,2139


In [ ]:
# routes.shape

(23, 9)

In [ ]:
# routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,4300,1,1,Juan Tabo,NaN,3,NaN,E17F38,000000
1,4312,1,2,Eubank-Ventura,NaN,3,NaN,DC5D2E,000000
2,4321,1,5,Montgomery/Carlisle,NaN,3,NaN,FF4E21,000000
3,4333,1,8,Menaul,NaN,3,NaN,7B097B,FFFFFF
4,4301,1,10,North Fourth Street,NaN,3,NaN,C30000,FFFFFF


In [ ]:
# routes.describe(include='all')

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
count,23,23,23,23,0,23,0,23,23
unique,23,1,23,23,0,1,0,22,2
top,4300,1,1,Juan Tabo,NaN,3,NaN,00FF00,FFFFFF
freq,1,23,1,1,NaN,23,NaN,2,16


In [ ]:
#✅ Step 3b: Merge with trips.txt to Get route_id
# Take all of df_clean and add on data from trips if the trip_id matches across both dataframes.
df_with_trips = df_clean.merge(trips, on="trip_id", how="left")


In [ ]:
# df_with_trips.head()

,snapshot_id,timestamp_collected,vehicle_id,latitude,longitude,heading,speed_mph,route_short_name,trip_id,next_stop_id,...,next_stop_sched_time,route_id,service_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,1,2025-07-22 11:17:02.268350,383,35.058513,-106.56991,17.0,0.0,16,601256,2801,...,15:18:45,4309,175.0.3,Louisiana & Gibson,NaN,1,53541,17162,1,1
1,2,2025-07-22 11:17:32.471202,383,35.058513,-106.56991,17.0,0.0,16,601256,2801,...,15:18:45,4309,175.0.3,Louisiana & Gibson,NaN,1,53541,17162,1,1
2,3,2025-07-22 11:18:02.682016,383,35.058513,-106.56991,17.0,0.0,16,601256,2801,...,15:18:45,4309,175.0.3,Louisiana & Gibson,NaN,1,53541,17162,1,1
3,4,2025-07-22 11:18:32.889178,383,35.058513,-106.56991,17.0,0.0,16,601256,2801,...,15:18:45,4309,175.0.3,Louisiana & Gibson,NaN,1,53541,17162,1,1
4,5,2025-07-22 11:19:03.095616,383,35.058513,-106.56991,17.0,0.0,16,601256,2801,...,15:18:45,4309,175.0.3,Louisiana & Gibson,NaN,1,53541,17162,1,1


In [ ]:
# df_with_trips.columns

Index(['snapshot_id', 'timestamp_collected', 'vehicle_id', 'latitude',
       'longitude', 'heading', 'speed_mph', 'route_short_name', 'trip_id',
       'next_stop_id', 'next_stop_name', 'next_stop_sched_time', 'route_id',
       'service_id', 'trip_headsign', 'trip_short_name', 'direction_id',
       'block_id', 'shape_id', 'wheelchair_accessible', 'bikes_allowed'],
      dtype='object')

####Merge check
1. Trip-Based Merges (Correct in Isolation)
- df_with_trips = df_clean.merge(trips, on="trip_id", how="left")
- df_full = df_with_trips.merge(routes, on="route_id", how="left")

✅ Good practice: merges are on appropriate keys (trip_id, route_id), and types are cast explicitly.

In [ ]:
# trips.columns

Index(['route_id', 'service_id', 'trip_id', 'trip_headsign', 'trip_short_name',
       'direction_id', 'block_id', 'shape_id', 'wheelchair_accessible',
       'bikes_allowed'],
      dtype='object')

In [ ]:
# routes.columns

Index(['route_id', 'agency_id', 'route_short_name', 'route_long_name',
       'route_desc', 'route_type', 'route_url', 'route_color',
       'route_text_color'],
      dtype='object')

In [ ]:
# routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,4300,1,1,Juan Tabo,NaN,3,NaN,E17F38,000000
1,4312,1,2,Eubank-Ventura,NaN,3,NaN,DC5D2E,000000
2,4321,1,5,Montgomery/Carlisle,NaN,3,NaN,FF4E21,000000
3,4333,1,8,Menaul,NaN,3,NaN,7B097B,FFFFFF
4,4301,1,10,North Fourth Street,NaN,3,NaN,C30000,FFFFFF


In [ ]:
#This cell caused some issues in the merge because the df_with_trips and routes dfs did not agree on route_short_name, so we are going to take another approach
# #✅ Step 4b: Merge with routes.txt to Get Descriptive Info
# df_with_trips["route_id"] = df_with_trips["route_id"].astype(str)

# # Ensure consistent types
# trips["shape_id"] = trips["shape_id"].astype(str)
# routes["route_id"] = routes["route_id"].astype(str)

# # Take all of df_with_trips and add on data from routes if the route_id matches across both dataframes.
# df_full = df_with_trips.merge(routes, on="route_id", how="left")


In [ ]:
#Let's avoid the route_..._x and route_..._y situation on the merge:
#✅ Step 4b: Merge with routes.txt to Get Descriptive Info
df_with_trips["route_id"] = df_with_trips["route_id"].astype(str)

# Ensure consistent types
trips["shape_id"] = trips["shape_id"].astype(str)
routes["route_id"] = routes["route_id"].astype(str)

# Assume 'df_with_trips' and 'routes' both contain 'route_id' and 'route_short_name'

# Step 1: Check 1:1 match on overlapping column
merged_check = df_with_trips[['route_id', 'route_short_name']].merge(
    routes[['route_id', 'route_short_name']],
    on='route_id',
    how='inner',
    suffixes=('_df_with_trips', '_routes')
)


In [ ]:
# merged_check.head()

,route_id,route_short_name_df_with_trips,route_short_name_routes
0,4309,16,16
1,4309,16,16
2,4309,16,16
3,4309,16,16
4,4309,16,16


In [ ]:
# merged_check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171355 entries, 0 to 171354
Data columns (total 3 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   route_id                        171355 non-null  object
 1   route_short_name_df_with_trips  171355 non-null  object
 2   route_short_name_routes         171355 non-null  object
dtypes: object(3)
memory usage: 3.9+ MB


In [ ]:
# Step 2: Find any mismatches
mismatch = merged_check[merged_check['route_short_name_df_with_trips'] != merged_check['route_short_name_routes']]

if not mismatch.empty:
    raise ValueError(f"Mismatch found in route_short_name across dataframes:\n{mismatch}")

ValueError: Mismatch found in route_short_name across dataframes:
       route_id route_short_name_df_with_trips route_short_name_routes
1331       4301                              5                      10
1359       4301                              5                      10
1387       4301                              5                      10
1415       4301                              5                      10
1443       4301                              5                      10
...         ...                            ...                     ...
171258     4330                            777                     766
171301     4331                            766                     777
171303     4330                            777                     766
171345     4331                            766                     777
171347     4330                            777                     766

[4995 rows x 3 columns]

In [ ]:
#Okay... so there are some mismatches between the data and the static routes info; let's see what they are
mismatch_summary = (
    mismatch[['route_id', 'route_short_name_df_with_trips', 'route_short_name_routes']]
    .drop_duplicates()
    .sort_values('route_id')
)
display(mismatch_summary)


,route_id,route_short_name_df_with_trips,route_short_name_routes
1331,4301,5,10
4168,4305,141,140
12535,4306,140,141
31112,4319,50,36
1628,4321,10,5
167163,4321,8,5
14006,4330,777,766
12187,4331,766,777
6894,4333,11,8


#This is the stopping point for 250731-1200; pick up here

In [ ]:
# [k for k, v in globals().items() if str(type(v)) == "<class 'pandas.core.frame.DataFrame'>"]


['_',
 '__',
 '___',
 'captured_data',
 'cd_df',
 'rows_with_same_values',
 'rows_without_same_values',
 'cd_df_copy',
 'cd_df_filtered',
 'df_new',
 'df_clean_test',
 'df_clean_trips',
 'df_clean',
 '_19',
 'trips',
 'routes',
 'df_with_trips',
 'merged_check',
 '_23',
 'mismatch',
 'mismatch_summary',
 '_28',
 '_29']

In [ ]:
# routes.head()


,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,4300,1,1,Juan Tabo,NaN,3,NaN,E17F38,000000
1,4312,1,2,Eubank-Ventura,NaN,3,NaN,DC5D2E,000000
2,4321,1,5,Montgomery/Carlisle,NaN,3,NaN,FF4E21,000000
3,4333,1,8,Menaul,NaN,3,NaN,7B097B,FFFFFF
4,4301,1,10,North Fourth Street,NaN,3,NaN,C30000,FFFFFF


In [ ]:
# routes.columns

Index(['route_id', 'agency_id', 'route_short_name', 'route_long_name',
       'route_desc', 'route_type', 'route_url', 'route_color',
       'route_text_color'],
      dtype='object')

In [ ]:
# Rename to preserve both versions of route_short_name
routes_renamed = routes.rename(columns={'route_short_name': 'route_short_name_static'})

# Merge with df_with_trips, keeping the RT version as primary
merged = df_with_trips.merge(routes_renamed, on='route_id', how='left')


In [ ]:
# merged.columns

Index(['snapshot_id', 'timestamp_collected', 'vehicle_id', 'latitude',
       'longitude', 'heading', 'speed_mph', 'route_short_name', 'trip_id',
       'next_stop_id', 'next_stop_name', 'next_stop_sched_time', 'route_id',
       'service_id', 'trip_headsign', 'trip_short_name', 'direction_id',
       'block_id', 'shape_id', 'wheelchair_accessible', 'bikes_allowed',
       'agency_id', 'route_short_name_static', 'route_long_name', 'route_desc',
       'route_type', 'route_url', 'route_color', 'route_text_color'],
      dtype='object')

In [ ]:
df_full = merged

In [ ]:
# df_full.describe(include='all')

,snapshot_id,timestamp_collected,vehicle_id,latitude,longitude,heading,speed_mph,route_short_name_x,trip_id,next_stop_id,...,wheelchair_accessible,bikes_allowed,agency_id,route_short_name_y,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
count,171355.000000,171355,171355.000000,171355.000000,171355.000000,171355.000000,171355.000000,171355,171355,171355,...,171355,171355,171355,171355,171355,0,171355,0,171355,171355
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,859,1792,...,1,1,1,22,22,0,1,0,21,2
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,777,601256,2801,...,1,1,1,777,ART Green Line,NaN,3,NaN,00FF00,FFFFFF
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16216,3228,3389,...,171355,171355,171355,16343,16343,NaN,171355,NaN,19281,106696
mean,61.079595,2025-07-23 06:57:32.870860544,899.873018,35.092129,-106.617839,171.332911,12.037279,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,2025-07-22 11:17:02.268350,353.000000,34.947920,-106.755080,0.000000,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,31.000000,2025-07-22 18:31:01.974678016,384.000000,35.074300,-106.657330,90.000000,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,61.000000,2025-07-23 12:25:57.993828096,614.000000,35.083379,-106.621956,180.000000,6.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,91.000000,2025-07-23 19:00:52.846295040,972.000000,35.108041,-106.569910,271.000000,24.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,120.000000,2025-07-24 01:13:30.399545,4102.000000,35.209220,-106.489108,359.000000,68.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# #✅ Step 5b: Load shapes.txt
# shapes = pd.read_csv("/content/gtfs_static/shapes.txt", dtype={"shape_id": str})

In [ ]:
# shapes.describe(include='all')

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
count,17005,17005.000000,17005.000000,17005.000000,17005.000000
unique,67,NaN,NaN,NaN,NaN
top,17160,NaN,NaN,NaN,NaN
freq,638,NaN,NaN,NaN,NaN
mean,NaN,35.100709,-106.633711,165.080094,9.988506
std,NaN,0.043697,0.063165,129.194960,7.095473
min,NaN,34.950369,-106.755186,1.000000,0.000000
25%,NaN,35.072826,-106.682325,64.000000,4.191400
50%,NaN,35.090442,-106.646916,134.000000,8.807300
75%,NaN,35.130635,-106.586336,237.000000,14.696600


In [ ]:
# shapes.head(1000)

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,17128,35.153625,-106.524465,1,0.0000
1,17128,35.152615,-106.524616,2,0.1137
2,17128,35.153623,-106.524464,3,0.1137
3,17128,35.154353,-106.524518,4,0.1949
4,17128,35.154264,-106.523311,5,0.3043
...,...,...,...,...,...
995,17134,35.086914,-106.520405,15,1.3658
996,17134,35.086921,-106.521217,16,1.4398
997,17134,35.086929,-106.521415,17,1.4578
998,17134,35.086928,-106.522050,18,1.5148


In [ ]:
#✅ Step 5b: Load shapes.txt
shapes = pd.read_csv("/content/gtfs_static/shapes.txt", dtype={"shape_id": str})

# Build LineStrings for each shape_id
shape_lines = {}
for shape_id, group in shapes.groupby("shape_id"):
    sorted_group = group.sort_values("shape_pt_sequence")
    coords = list(zip(sorted_group["shape_pt_lon"], sorted_group["shape_pt_lat"]))
    shape_lines[shape_id] = LineString(coords)

# Merge trips and routes to link shape_id to route_short_name
shape_route_map = (
    trips.merge(routes, on="route_id", how="left")
         .dropna(subset=["route_short_name"])
         .drop_duplicates(subset=["shape_id"])
         .set_index("shape_id")[["route_id", "route_short_name"]]
)

In [ ]:
#✅ Step 6b: Create a data_quality Flag
def classify_row(row):
    if row["trip_id"] in invalid_trip_ids:
        return "invalid_trip_id"
    elif pd.isna(row["route_id"]):
        return "missing_route_id"
    elif pd.isna(row["route_long_name"]):
        return "missing_route_metadata"
    else:
        return "valid"

df_full["data_quality"] = df_full.apply(classify_row, axis=1)
print(df_full["data_quality"].value_counts())


data_quality
valid    171355
Name: count, dtype: int64


#####Possible Deprication

In [ ]:
#Not sure if this is needed
# #Load the trips, routes, shapes from Static data
# with zipfile.ZipFile(gtfs_zip_path, 'r') as z:
#     trips_df = pd.read_csv(z.open("trips.txt"))
#     routes_df = pd.read_csv(z.open("routes.txt"))
#     # Load GTFS shapes.txt into a DataFrame
#     shapes_df = pd.read_csv(z.open("shapes.txt"))

#Clean the RT data

In [ ]:
#🔹 1.1 Filter for Valid Rows
df_valid = df_full[df_full["data_quality"] == "valid"].copy()

#🔹 1.2 Parse Timestamps
df_valid["timestamp"] = pd.to_datetime(df_valid["timestamp_collected"], utc=True)

#🔹 1.3 Sort by Vehicle and Timestamp
df_valid = df_valid.sort_values(by=["vehicle_id", "timestamp"])

#🔹 1.4 Organize by Vehicle
#This creates a dictionary keyed by vehicle ID, each with a sorted DataFrame:
vehicle_groups = dict(tuple(df_valid.groupby("vehicle_id")))

#You can confirm how many distinct vehicles you’re tracking:
print("Vehicle count:", len(vehicle_groups))


Vehicle count: 87


In [ ]:
# df_full.shape

(171355, 30)

In [ ]:
# df_full.describe(include='all')

,snapshot_id,timestamp_collected,vehicle_id,latitude,longitude,heading,speed_mph,route_short_name_x,trip_id,next_stop_id,...,bikes_allowed,agency_id,route_short_name_y,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,data_quality
count,171355.000000,171355,171355.000000,171355.000000,171355.000000,171355.000000,171355.000000,171355,171355,171355,...,171355,171355,171355,171355,0,171355,0,171355,171355,171355
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,859,1792,...,1,1,22,22,0,1,0,21,2,1
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,777,601256,2801,...,1,1,777,ART Green Line,NaN,3,NaN,00FF00,FFFFFF,valid
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16216,3228,3389,...,171355,171355,16343,16343,NaN,171355,NaN,19281,106696,171355
mean,61.079595,2025-07-23 06:57:32.870860544,899.873018,35.092129,-106.617839,171.332911,12.037279,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,2025-07-22 11:17:02.268350,353.000000,34.947920,-106.755080,0.000000,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,31.000000,2025-07-22 18:31:01.974678016,384.000000,35.074300,-106.657330,90.000000,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,61.000000,2025-07-23 12:25:57.993828096,614.000000,35.083379,-106.621956,180.000000,6.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,91.000000,2025-07-23 19:00:52.846295040,972.000000,35.108041,-106.569910,271.000000,24.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,120.000000,2025-07-24 01:13:30.399545,4102.000000,35.209220,-106.489108,359.000000,68.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# df_full.head()

,snapshot_id,timestamp_collected,vehicle_id,latitude,longitude,heading,speed_mph,route_short_name_x,trip_id,next_stop_id,...,bikes_allowed,agency_id,route_short_name_y,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,data_quality
0,1,2025-07-22 11:17:02.268350,383,35.058513,-106.56991,17.0,0.0,16,601256,2801,...,1,1,16,University/Gibson/Broadway,NaN,3,NaN,C01D8C,FFFFFF,valid
1,2,2025-07-22 11:17:32.471202,383,35.058513,-106.56991,17.0,0.0,16,601256,2801,...,1,1,16,University/Gibson/Broadway,NaN,3,NaN,C01D8C,FFFFFF,valid
2,3,2025-07-22 11:18:02.682016,383,35.058513,-106.56991,17.0,0.0,16,601256,2801,...,1,1,16,University/Gibson/Broadway,NaN,3,NaN,C01D8C,FFFFFF,valid
3,4,2025-07-22 11:18:32.889178,383,35.058513,-106.56991,17.0,0.0,16,601256,2801,...,1,1,16,University/Gibson/Broadway,NaN,3,NaN,C01D8C,FFFFFF,valid
4,5,2025-07-22 11:19:03.095616,383,35.058513,-106.56991,17.0,0.0,16,601256,2801,...,1,1,16,University/Gibson/Broadway,NaN,3,NaN,C01D8C,FFFFFF,valid


In [ ]:
# df_valid.shape

(171355, 31)

In [ ]:
# df_valid.describe(include='all')

,snapshot_id,timestamp_collected,vehicle_id,latitude,longitude,heading,speed_mph,route_short_name_x,trip_id,next_stop_id,...,agency_id,route_short_name_y,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,data_quality,timestamp
count,171355.000000,171355,171355.000000,171355.000000,171355.000000,171355.000000,171355.000000,171355,171355,171355,...,171355,171355,171355,0,171355,0,171355,171355,171355,171355
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,859,1792,...,1,22,22,0,1,0,21,2,1,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,777,601256,2801,...,1,777,ART Green Line,NaN,3,NaN,00FF00,FFFFFF,valid,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16216,3228,3389,...,171355,16343,16343,NaN,171355,NaN,19281,106696,171355,NaN
mean,61.079595,2025-07-23 06:57:32.870860288,899.873018,35.092129,-106.617839,171.332911,12.037279,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-23 06:57:32.870860288+00:00
min,1.000000,2025-07-22 11:17:02.268350,353.000000,34.947920,-106.755080,0.000000,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-22 11:17:02.268350+00:00
25%,31.000000,2025-07-22 18:31:01.974678016,384.000000,35.074300,-106.657330,90.000000,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-22 18:31:01.974678016+00:00
50%,61.000000,2025-07-23 12:25:57.993828096,614.000000,35.083379,-106.621956,180.000000,6.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-23 12:25:57.993828096+00:00
75%,91.000000,2025-07-23 19:00:52.846295040,972.000000,35.108041,-106.569910,271.000000,24.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-23 19:00:52.846295040+00:00
max,120.000000,2025-07-24 01:13:30.399545,4102.000000,35.209220,-106.489108,359.000000,68.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-24 01:13:30.399545+00:00


In [ ]:
# df_valid.head(1000)

,snapshot_id,timestamp_collected,vehicle_id,latitude,longitude,heading,speed_mph,route_short_name_x,trip_id,next_stop_id,...,agency_id,route_short_name_y,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,data_quality,timestamp
84264,84,2025-07-23 11:57:44.872302,353,35.206084,-106.660050,125.0,9.0,155,601002,704230731,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 11:57:44.872302+00:00
84278,85,2025-07-23 11:58:15.130567,353,35.207074,-106.659887,277.0,17.0,155,601002,704230731,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 11:58:15.130567+00:00
84292,86,2025-07-23 11:58:45.330622,353,35.207163,-106.661616,273.0,1.0,155,601002,704230731,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 11:58:45.330622+00:00
84307,87,2025-07-23 11:59:15.571828,353,35.206109,-106.661820,184.0,0.0,155,601002,704230731,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 11:59:15.571828+00:00
84324,88,2025-07-23 11:59:45.773193,353,35.205764,-106.661329,96.0,23.0,155,601002,704230731,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 11:59:45.773193+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137260,3,2025-07-23 20:15:13.945586,353,35.047953,-106.708683,197.0,41.0,155,601016,704230685,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 20:15:13.945586+00:00
137319,4,2025-07-23 20:15:44.186966,353,35.042688,-106.710668,198.0,13.0,155,601016,3083,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 20:15:44.186966+00:00
137378,5,2025-07-23 20:16:14.424263,353,35.042103,-106.710826,194.0,25.0,155,601016,3083,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 20:16:14.424263+00:00
137437,6,2025-07-23 20:16:44.663948,353,35.037412,-106.712242,190.0,2.0,155,601016,704230818,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 20:16:44.663948+00:00


#Anomly Detection

🔹 1. Detect Jumps and Gaps

Already implemented, but here’s the modular form:

In [ ]:
# # 🔹 1. Detect Jumps and Gaps
# def detect_jumps_and_gaps(df, params):
#     """
#     Detects jump or gap anomalies based on distance (meters) and time (seconds)
#     thresholds pulled from the anomaly_params dictionary.

#     Parameters:
#         df (pd.DataFrame): Filtered DataFrame for a single vehicle
#         params (dict): Anomaly detection thresholds from widget interface

#     Returns:
#         List[dict]: List of detected jump or gap anomalies
#     """
#     distance_threshold = params["JUMP_DISTANCE_METERS"]           # in meters
#     time_threshold = params["DISAPPEARANCE_TIME_THRESHOLD"]       # in seconds

#     anomalies = []
#     for i in range(1, len(df)):
#         row_prev, row_curr = df.iloc[i - 1], df.iloc[i]
#         time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()

#         distance = geodesic(
#             (row_prev["latitude"], row_prev["longitude"]),
#             (row_curr["latitude"], row_curr["longitude"])
#         ).meters

#         if time_diff > time_threshold or distance > distance_threshold:
#             anomalies.append({
#                 "vehicle_id": row_curr["vehicle_id"],
#                 "timestamp_prev": row_prev["timestamp"],
#                 "timestamp_curr": row_curr["timestamp"],
#                 "time_diff_sec": time_diff,
#                 "distance_m": distance,
#                 "is_gap": time_diff > time_threshold,
#                 "is_jump": distance > distance_threshold,
#                 "anomaly_type": "jump_or_gap"
#             })

#     return anomalies
# #Note: changed row["timestamp_collected"] to row["timestamp"] throughout, assuming that we've already converted timestamp_collected to UTC and assigned it to a new "timestamp" column at the start of the anomaly pipeline (which the existing pipeline does). This avoids inconsistency.

In [ ]:
# 🔹 1. Detect Jumps and Gaps – WITHIN trip_id
def detect_jumps_and_gaps(df, params):
    """
     Detects jump or gap anomalies based on distance (meters) and time (seconds)
     thresholds pulled from the anomaly_params dictionary.

    Parameters:
        df (pd.DataFrame): Filtered DataFrame for a single vehicle, sorted by timestamp.
                           Must include 'trip_id', 'timestamp', 'latitude', 'longitude'.
        params (dict): Anomaly detection thresholds from widget interface.

    Returns:
        List[dict]: Detected jump or gap anomalies.
    """
    distance_threshold = params["JUMP_DISTANCE_METERS"]           # meters
    time_threshold = params["DISAPPEARANCE_TIME_THRESHOLD"]       # seconds

    anomalies = []
    for i in range(1, len(df)):
        row_prev, row_curr = df.iloc[i - 1], df.iloc[i]

        # 💡 Trip boundary check — skip if different trip
        if row_prev["trip_id"] != row_curr["trip_id"]:
            continue

        time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()

        distance = geodesic(
            (row_prev["latitude"], row_prev["longitude"]),
            (row_curr["latitude"], row_curr["longitude"])
        ).meters

        if time_diff > time_threshold or distance > distance_threshold:
            anomalies.append({
                "vehicle_id": row_curr["vehicle_id"],
                "trip_id": row_curr["trip_id"],  # optional but useful for tracing
                "timestamp_prev": row_prev["timestamp"],
                "timestamp_curr": row_curr["timestamp"],
                "time_diff_sec": time_diff,
                "distance_m": distance,
                "is_gap": time_diff > time_threshold,
                "is_jump": distance > distance_threshold,
                "anomaly_type": "jump_or_gap"
            })

    return anomalies


In [ ]:
# #2. Detect Stuck Vehicles
# def detect_stuck_vehicle(df, params):
#     """
#     Detects stuck vehicles using a rolling window of speed and spatial consistency,
#     segmented by trip_id to avoid cross-trip contamination.

#     Parameters:
#         df (pd.DataFrame): Must contain 'speed_mph', 'latitude', 'longitude', and 'trip_id'.
#         params (dict): Widget-defined anomaly thresholds.

#     Returns:
#         pd.DataFrame: Rows flagged as 'stuck_vehicle'.
#     """
#     speed_thresh = params.get("STUCK_SPEED_MPH", 1.0)   # Default: 1 mph
#     window = params.get("STUCK_WINDOW_SIZE", 4)         # Default: 4-frame window

#     stuck_flags = (
#         (df["speed_mph"].rolling(window).mean() < speed_thresh) &
#         (df["latitude"].diff().abs().rolling(window).mean() < 0.0001) &
#         (df["longitude"].diff().abs().rolling(window).mean() < 0.0001)
#     )

#     return df[stuck_flags.fillna(False)].assign(anomaly_type="stuck_vehicle")


In [ ]:
# 🔹 2. Detect Stuck Vehicles
def detect_stuck_vehicle(df, params):
    """
    Detects stuck vehicles using a rolling window of speed and spatial consistency,
    segmented by trip_id to avoid cross-trip contamination.

    Parameters:
        df (pd.DataFrame): Must contain 'speed_mph', 'latitude', 'longitude', and 'trip_id'.
        params (dict): Widget-defined anomaly thresholds.

    Returns:
        pd.DataFrame: Flagged stuck vehicle anomalies.
    """
    speed_thresh = params.get("STUCK_SPEED_MPH", 1.0)   # mph
    window = params.get("STUCK_WINDOW_SIZE", 4)         # frames

    flagged_list = []

    for trip_id, group in df.groupby("trip_id"):
        group = group.sort_values("timestamp").copy()

        # Compute rolling conditions
        speed_ok = group["speed_mph"].rolling(window).mean() < speed_thresh
        lat_ok = group["latitude"].diff().abs().rolling(window).mean() < 0.0001
        lon_ok = group["longitude"].diff().abs().rolling(window).mean() < 0.0001

        stuck_flags = speed_ok & lat_ok & lon_ok

        flagged = group[stuck_flags.fillna(False)].copy()
        if not flagged.empty:
            flagged["anomaly_type"] = "stuck_vehicle"
            flagged_list.append(flagged)

    return pd.concat(flagged_list, ignore_index=True) if flagged_list else pd.DataFrame(columns=df.columns.tolist() + ["anomaly_type"])


In [ ]:
# # 🔹 3. Detect Impossible Speeds
# def detect_impossible_speeds(df, params):
#     """
#     Detects movement segments where computed speed exceeds threshold.

#     Parameters:
#         df (pd.DataFrame): Filtered or per-vehicle data
#         params (dict): Anomaly detection parameters from UI

#     Returns:
#         pd.DataFrame: Rows with flagged impossible-speed anomalies
#     """
#     # Convert mph to kph for geodesic-based calc
#     speed_limit_kph = params["SPEED_LIMIT_MPH"] * 1.60934

#     records = []
#     for i in range(1, len(df)):
#         row_prev, row_curr = df.iloc[i - 1], df.iloc[i]
#         time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()
#         if time_diff == 0:
#             continue

#         distance = geodesic(
#             (row_prev["latitude"], row_prev["longitude"]),
#             (row_curr["latitude"], row_curr["longitude"])
#         ).meters

#         speed_kph = (distance / time_diff) * 3.6  # m/s → km/h

#         if speed_kph > speed_limit_kph:
#             records.append({
#                 "vehicle_id": row_curr["vehicle_id"],
#                 "timestamp_curr": row_curr["timestamp"],
#                 "computed_speed_kph": speed_kph,
#                 "distance_m": distance,
#                 "anomaly_type": "impossible_speed"
#             })

#     return pd.DataFrame(records)


In [ ]:
# 🔹 3. Detect Impossible Speeds
def detect_impossible_speeds(df, params):
    """
    Detects movement segments where computed speed exceeds threshold,
    segmented by trip_id to avoid cross-trip errors.

    Parameters:
        df (pd.DataFrame): Must contain timestamp, lat/lon, trip_id
        params (dict): Anomaly detection thresholds

    Returns:
        pd.DataFrame: Flagged rows with 'impossible_speed'
    """
    speed_limit_kph = params["SPEED_LIMIT_MPH"] * 1.60934
    output_rows = []

    for trip_id, group in df.groupby("trip_id"):
        group = group.sort_values("timestamp").copy()

        for i in range(1, len(group)):
            row_prev, row_curr = group.iloc[i - 1], group.iloc[i]
            time_diff = (row_curr["timestamp"] - row_prev["timestamp"]).total_seconds()
            if time_diff <= 0:
                continue

            distance = geodesic(
                (row_prev["latitude"], row_prev["longitude"]),
                (row_curr["latitude"], row_curr["longitude"])
            ).meters

            speed_kph = (distance / time_diff) * 3.6

            if speed_kph > speed_limit_kph:
                output_rows.append({
                    "vehicle_id": row_curr["vehicle_id"],
                    "trip_id": trip_id,
                    "timestamp_curr": row_curr["timestamp"],
                    "computed_speed_kph": speed_kph,
                    "distance_m": distance,
                    "anomaly_type": "impossible_speed"
                })

    return pd.DataFrame(output_rows)


In [ ]:
# # 🔹 4. Detect Backtracking (Heading Reversal)
# def detect_backtracking(df, params):
#     """
#     Detects heading reversals suggesting backtracking behavior.

#     Parameters:
#         df (pd.DataFrame): Filtered or per-vehicle data
#         params (dict): Anomaly detection parameters from UI

#     Returns:
#         pd.DataFrame: Rows where heading reversed beyond threshold
#     """
#     reversal_thresh = params["REVERSAL_HEADING_THRESHOLD"]

#     if "heading" not in df.columns:
#         return pd.DataFrame()

#     heading_diff = df["heading"].diff().abs()
#     backtrack_flags = heading_diff.between(reversal_thresh, 200)

#     return df[backtrack_flags.fillna(False)].assign(anomaly_type="backtracking")


In [ ]:
# 🔹 4. Detect Backtracking (Heading Reversal)
def detect_backtracking(df, params):
    """
    Detects heading reversals (backtracking) within trip boundaries only,
    with wraparound-safe angle diffing and traceability.

    Parameters:
        df (pd.DataFrame): Filtered per-vehicle data with heading + trip_id
        params (dict): Anomaly detection thresholds

    Returns:
        pd.DataFrame: Rows flagged as 'backtracking'
    """
    reversal_thresh = params.get("REVERSAL_HEADING_THRESHOLD", 120)
    output_frames = []

    if "heading" not in df.columns or "trip_id" not in df.columns:
        return pd.DataFrame()

    for trip_id, group in df.groupby("trip_id"):
        group = group.sort_values("timestamp").copy()

        # Compute heading difference with wraparound correction
        group["heading_prev"] = group["heading"].shift()
        group["heading_diff"] = (group["heading"] - group["heading_prev"]).abs()
        group["heading_diff"] = group["heading_diff"].apply(
            lambda x: min(x, 360 - x) if pd.notna(x) else x
        )

        # Identify backtracking
        backtrack_flags = group["heading_diff"].between(reversal_thresh, 200)

        flagged = group[backtrack_flags.fillna(False)].copy()
        flagged["anomaly_type"] = "backtracking"
        flagged["timestamp_prev"] = group["timestamp"].shift()

        output_frames.append(flagged)

    return pd.concat(output_frames, ignore_index=True) if output_frames else pd.DataFrame()



In [ ]:
# # 🔹 5. Detect Repeated Points
# def detect_repeated_points(df, params):
#     """
#     Detects repeated GPS coordinates (vehicle not moving).

#     Parameters:
#         df (pd.DataFrame): Filtered or per-vehicle snapshot data
#         params (dict): Anomaly detection parameters from UI

#     Returns:
#         pd.DataFrame: Rows flagged as having repeated lat/lon values
#     """
#     tolerance = abs(params.get("REPEATED_COORD_TOLERANCE", 1e-5))

#     if "latitude" not in df.columns or "longitude" not in df.columns:
#         return pd.DataFrame()

#     repeated = (
#         (df["latitude"].diff().abs() < tolerance) &
#         (df["longitude"].diff().abs() < tolerance)
#     )

#     return df[repeated.fillna(False)].assign(anomaly_type="repeated_points")


In [ ]:
# 🔹 5. Detect Repeated Points
def detect_repeated_points(df, params):
    """
    Detects repeated GPS coordinates (vehicle not moving), scoped to each trip.

    Parameters:
        df (pd.DataFrame): Per-vehicle GPS snapshot data
        params (dict): UI-provided anomaly detection thresholds

    Returns:
        pd.DataFrame: Rows flagged as 'repeated_points'
    """
    tolerance = abs(params.get("REPEATED_COORD_TOLERANCE", 1e-5))
    output = []

    if "latitude" not in df.columns or "longitude" not in df.columns or "trip_id" not in df.columns:
        return pd.DataFrame()

    for trip_id, group in df.groupby("trip_id"):
        group = group.sort_values("timestamp").copy()
        lat_repeat = group["latitude"].diff().abs() < tolerance
        lon_repeat = group["longitude"].diff().abs() < tolerance
        repeated = lat_repeat & lon_repeat

        flagged = group[repeated.fillna(False)].copy()
        flagged["anomaly_type"] = "repeated_points"
        output.append(flagged)

    return pd.concat(output, ignore_index=True) if output else pd.DataFrame()


In [ ]:
# # 🔹 6. Detect Disappearance Without Return
# def detect_disappeared(df, snapshot_end_time, params):
#     """
#     Detects vehicles that have not reappeared by the end of the snapshot period.

#     Parameters:
#         df (pd.DataFrame): Data for a single vehicle
#         snapshot_end_time (datetime): End of data collection
#         params (dict): Anomaly detection thresholds from widget

#     Returns:
#         pd.DataFrame: Single-row disappearance anomaly, or empty DataFrame
#     """
#     if df.empty:
#         return pd.DataFrame()

#     last_seen = df["timestamp"].max()
#     time_gap_sec = (snapshot_end_time - last_seen).total_seconds()
#     disappearance_thresh = params["DISAPPEARANCE_TIME_THRESHOLD"]

#     if time_gap_sec > disappearance_thresh:
#         return pd.DataFrame([{
#             "vehicle_id": df["vehicle_id"].iloc[0],
#             "last_seen": last_seen,
#             "anomaly_type": "disappearance"
#         }])

#     return pd.DataFrame()


In [ ]:
# 🔹 6. Detect Disappearance Without Return
def detect_disappeared(df, snapshot_end_time, params):
    """
    Detects disappearance per trip_id — if the trip ends and never resumes.

    Parameters:
        df (pd.DataFrame): Data for a single vehicle
        snapshot_end_time (datetime): End of snapshot
        params (dict): Anomaly config

    Returns:
        pd.DataFrame: One row per disappeared trip, if any
    """
    disappearance_thresh = params["DISAPPEARANCE_TIME_THRESHOLD"]
    output = []

    for trip_id, group in df.groupby("trip_id"):
        last_seen = group["timestamp"].max()
        time_gap_sec = (snapshot_end_time - last_seen).total_seconds()

        if time_gap_sec > disappearance_thresh:
            output.append({
                "vehicle_id": group["vehicle_id"].iloc[0],
                "trip_id": trip_id,
                "last_seen": last_seen,
                "anomaly_type": "disappearance"
            })

    return pd.DataFrame(output) if output else pd.DataFrame()


In [ ]:
# 🔹 7. Detect Early Appearance
def detect_early_appearance(df, snapshot_start_time, params):
    """
    Detects vehicles that appear too early (likely pre-start ghost data).

    Parameters:
        df (pd.DataFrame): Data for a single vehicle
        snapshot_start_time (datetime): Start of capture window
        params (dict): Anomaly detection thresholds from widget

    Returns:
        pd.DataFrame: Single-row anomaly, or empty DataFrame
    """
    margin_seconds = params.get("EARLY_APPEARANCE_MARGIN_SEC", 30)
    first_seen = df["timestamp"].min()
    delta = (first_seen - snapshot_start_time).total_seconds()

    if delta < margin_seconds:
        return pd.DataFrame([{
            "vehicle_id": df["vehicle_id"].iloc[0],
            "first_seen": first_seen,
            "anomaly_type": "early_appearance"
        }])
    return pd.DataFrame()


In [ ]:
# # 🔹 8. Detect Off-Route Movement
# def detect_off_route(df_vehicle, shape_lines, params):
#     """
#     Flags GPS points that are farther than OFF_ROUTE_BUFFER_M from the expected route shape.

#     Parameters:
#         df_vehicle (pd.DataFrame): All points for a single vehicle
#         shape_lines (dict): Dictionary of LineStrings per shape_id
#         params (dict): Thresholds including OFF_ROUTE_BUFFER_M

#     Returns:
#         pd.DataFrame: Off-route GPS points
#     """
#     buffer_m = params.get("OFF_ROUTE_BUFFER_M", 50)
#     records = []

#     for _, row in df_vehicle.iterrows():
#         shape_id = str(row.get("shape_id"))
#         if shape_id not in shape_lines:
#             continue  # shape not available

#         route_line = shape_lines[shape_id]
#         vehicle_point = Point(row["longitude"], row["latitude"])

#         # Project point onto shape line and compute geodesic distance
#         closest_point = route_line.interpolate(route_line.project(vehicle_point))
#         dist_m = geodesic(
#             (row["latitude"], row["longitude"]),
#             (closest_point.y, closest_point.x)
#         ).meters

#         if dist_m > buffer_m:
#             records.append({
#                 "vehicle_id": row["vehicle_id"],
#                 "timestamp": row["timestamp"],
#                 "route_short_name": row.get("route_short_name"),
#                 "distance_from_route_m": dist_m,
#                 "latitude": row["latitude"],
#                 "longitude": row["longitude"],
#                 "shape_id": shape_id,
#                 "anomaly_type": "off_route"
#             })

#     return pd.DataFrame(records)


In [ ]:
# 🔹 8. Detect Off-Route Movement
def detect_off_route(df_vehicle, shape_lines, params):
    """
    Detect off-route anomalies using trip-level shape validation.

    Parameters:
        df_vehicle (pd.DataFrame): All rows for a single vehicle.
        shape_lines (dict): shape_id → LineString
        params (dict): Dictionary with OFF_ROUTE_BUFFER_M

    Returns:
        pd.DataFrame: Flagged off-route points
    """
    buffer_m = params.get("OFF_ROUTE_BUFFER_M", 50)
    records = []

    for trip_id, trip_df in df_vehicle.groupby("trip_id"):
        # Check if all rows share the same shape_id
        shape_ids = trip_df["shape_id"].dropna().unique()

        if len(shape_ids) != 1:
            # Ambiguous shape_id — skip this trip to avoid misflagging
            continue

        shape_id = str(shape_ids[0])
        if shape_id not in shape_lines:
            continue

        route_line = shape_lines[shape_id]

        for _, row in trip_df.iterrows():
            vehicle_point = Point(row["longitude"], row["latitude"])
            projected = route_line.interpolate(route_line.project(vehicle_point))
            dist_m = geodesic(
                (row["latitude"], row["longitude"]),
                (projected.y, projected.x)
            ).meters

            if dist_m > buffer_m:
                records.append({
                    "vehicle_id": row["vehicle_id"],
                    "trip_id": trip_id,
                    "timestamp": row["timestamp"],
                    "route_short_name": row.get("route_short_name"),
                    "distance_from_route_m": dist_m,
                    "latitude": row["latitude"],
                    "longitude": row["longitude"],
                    "shape_id": shape_id,
                    "anomaly_type": "off_route"
                })

    return pd.DataFrame(records)

# Note: ✅ Key Protections Added
# Fix	Explanation
# groupby("trip_id")	Ensures anomalies only detected within a single trip context
# unique shape_id check	Filters out data where shape mapping is ambiguous
# skip trip if shape_id missing or mismatched	Prevents false positives from misassigned shapes
# Includes trip_id in output	Improves downstream traceability and debugging

In [ ]:
# Define options
detector_options = [
    ("Jumps & Gaps", "jumpgap"),
    ("Stuck Vehicles", "stuck"),
    ("Impossible Speeds", "speed"),
    ("Backtracking", "backtrack"),
    ("Repeated Points", "repeated"),
    ("Disappearance", "disappear"),
    ("Early Appearance", "early"),
    ("Off-Route", "offroute")
]

# Create selector widget
detector_selector = widgets.SelectMultiple(
    options=detector_options,
    value=[k for _, k in detector_options],  # default: all selected
    description="Detectors:",
    layout=widgets.Layout(width="60%")
)

# Create button
confirm_button = widgets.Button(
    description="Confirm Selection",
    button_style='primary',
    icon='check'
)

# Output area
selection_output = widgets.Output()

# Define callback
def on_confirm_clicked(b):
    with selection_output:
        clear_output()
        selected = list(detector_selector.value)
        print("✅ Selected anomaly detectors:")
        for detector in selected:
            print(f" - {detector}")
        # Optionally set a global variable for later use:
        global selected_detectors
        selected_detectors = selected

# Link button to callback
confirm_button.on_click(on_confirm_clicked)

# Display widgets
display(widgets.VBox([detector_selector, confirm_button, selection_output]))


In [ ]:
# # I believe we can skip step 1 because it was already done earlier
# # # Step 1: Normalize timestamp and rebuild vehicle groups
# # df_valid["timestamp"] = pd.to_datetime(df_valid["timestamp_collected"], utc=True)
# # vehicle_groups = dict(tuple(df_valid.groupby("vehicle_id")))

# # Step 2: Detect each anomaly type using widget-supplied parameters
# jumpgap_records = []
# stuck_records = []
# speed_records = []
# backtrack_records = []
# repeated_records = []
# disappear_records = []
# early_records = []
# offroute_records = []

# snapshot_start = df_valid["timestamp"].min()
# snapshot_end = df_valid["timestamp"].max()

# for vehicle_id, df_vehicle in vehicle_groups.items():
#     df_vehicle = df_vehicle.sort_values("timestamp").reset_index(drop=True)

#     # 1. Jumps and Gaps
#     jumpgap_records.extend(detect_jumps_and_gaps(df_vehicle, params=anomaly_params))

#     # 2. Stuck Vehicles
#     stuck = detect_stuck_vehicle(df_vehicle, params=anomaly_params)
#     if not stuck.empty:
#         stuck_records.append(stuck)

#     # 3. Impossible Speeds
#     speed = detect_impossible_speeds(df_vehicle, params=anomaly_params)
#     if not speed.empty:
#         speed_records.append(speed)

#     # 4. Backtracking
#     backtrack = detect_backtracking(df_vehicle, params=anomaly_params)
#     if not backtrack.empty:
#         backtrack_records.append(backtrack)

#     # 5. Repeated Points
#     repeat = detect_repeated_points(df_vehicle, params=anomaly_params)
#     if not repeat.empty:
#         repeated_records.append(repeat)

#     # 6. Disappearance
#     disappear = detect_disappeared(df_vehicle, snapshot_end_time=snapshot_end, params=anomaly_params)
#     if not disappear.empty:
#         disappear_records.append(disappear)

#     # 7. Early Appearance
#     early = detect_early_appearance(df_vehicle, snapshot_start_time=snapshot_start, params=anomaly_params)
#     if not early.empty:
#         early_records.append(early)

#     # 8. Off-Route
#     offroute = detect_off_route(df_vehicle, shape_lines=shape_lines, params=anomaly_params)
#     if not offroute.empty:
#         offroute_records.append(offroute)

# # Step 3: Combine to DataFrames
# df_anomalies_jumpgap   = pd.DataFrame(jumpgap_records)
# df_anomalies_stuck     = pd.concat(stuck_records, ignore_index=True)     if stuck_records     else pd.DataFrame()
# df_anomalies_speed     = pd.concat(speed_records, ignore_index=True)     if speed_records     else pd.DataFrame()
# df_anomalies_backtrack = pd.concat(backtrack_records, ignore_index=True) if backtrack_records else pd.DataFrame()
# df_anomalies_repeated  = pd.concat(repeated_records, ignore_index=True)  if repeated_records  else pd.DataFrame()
# df_anomalies_disappear = pd.concat(disappear_records, ignore_index=True) if disappear_records else pd.DataFrame()
# df_anomalies_early     = pd.concat(early_records, ignore_index=True)     if early_records     else pd.DataFrame()
# df_anomalies_offroute  = pd.concat(offroute_records, ignore_index=True)  if offroute_records  else pd.DataFrame()

# # Step 4: Combine all anomalies into a single DataFrame
# anomaly_frames = [
#     df_anomalies_jumpgap,
#     df_anomalies_stuck,
#     df_anomalies_speed,
#     df_anomalies_backtrack,
#     df_anomalies_repeated,
#     df_anomalies_disappear,
#     df_anomalies_early,
#     df_anomalies_offroute
# ]

# anomaly_frames = [df for df in anomaly_frames if 'anomaly_type' in df.columns and not df.empty]
# df_anomalies_full = pd.concat(anomaly_frames, ignore_index=True)

# # Report
# print("✅ Unified anomaly count:", len(df_anomalies_full))
# display(df_anomalies_full["anomaly_type"].value_counts())


KeyboardInterrupt: 

In [ ]:
# Selected detectors from UI
selected_detectors = list(detector_selector.value)

# Reset result containers
jumpgap_records = []
stuck_records = []
speed_records = []
backtrack_records = []
repeated_records = []
disappear_records = []
early_records = []
offroute_records = []

snapshot_start = df_valid["timestamp"].min()
snapshot_end = df_valid["timestamp"].max()

# Run detectors conditionally
for vehicle_id, df_vehicle in vehicle_groups.items():
    df_vehicle = df_vehicle.sort_values("timestamp").reset_index(drop=True)

    if "jumpgap" in selected_detectors:
        jumpgap_records.extend(detect_jumps_and_gaps(df_vehicle, params=anomaly_params))

    if "stuck" in selected_detectors:
        stuck = detect_stuck_vehicle(df_vehicle, params=anomaly_params)
        if not stuck.empty:
            stuck_records.append(stuck)

    if "speed" in selected_detectors:
        speed = detect_impossible_speeds(df_vehicle, params=anomaly_params)
        if not speed.empty:
            speed_records.append(speed)

    if "backtrack" in selected_detectors:
        backtrack = detect_backtracking(df_vehicle, params=anomaly_params)
        if not backtrack.empty:
            backtrack_records.append(backtrack)

    if "repeated" in selected_detectors:
        repeat = detect_repeated_points(df_vehicle, params=anomaly_params)
        if not repeat.empty:
            repeated_records.append(repeat)

    if "disappear" in selected_detectors:
        disappear = detect_disappeared(df_vehicle, snapshot_end_time=snapshot_end, params=anomaly_params)
        if not disappear.empty:
            disappear_records.append(disappear)

    if "early" in selected_detectors:
        early = detect_early_appearance(df_vehicle, snapshot_start_time=snapshot_start, params=anomaly_params)
        if not early.empty:
            early_records.append(early)

    if "offroute" in selected_detectors:
        offroute = detect_off_route(df_vehicle, shape_lines=shape_lines, params=anomaly_params)
        if not offroute.empty:
            offroute_records.append(offroute)

# Step 3: Combine to DataFrames
df_anomalies_jumpgap   = pd.DataFrame(jumpgap_records)
df_anomalies_stuck     = pd.concat(stuck_records, ignore_index=True)     if stuck_records     else pd.DataFrame()
df_anomalies_speed     = pd.concat(speed_records, ignore_index=True)     if speed_records     else pd.DataFrame()
df_anomalies_backtrack = pd.concat(backtrack_records, ignore_index=True) if backtrack_records else pd.DataFrame()
df_anomalies_repeated  = pd.concat(repeated_records, ignore_index=True)  if repeated_records  else pd.DataFrame()
df_anomalies_disappear = pd.concat(disappear_records, ignore_index=True) if disappear_records else pd.DataFrame()
df_anomalies_early     = pd.concat(early_records, ignore_index=True)     if early_records     else pd.DataFrame()
df_anomalies_offroute  = pd.concat(offroute_records, ignore_index=True)  if offroute_records  else pd.DataFrame()

# Step 4: Combine all selected anomalies
anomaly_frames = []
if "jumpgap" in selected_detectors:   anomaly_frames.append(df_anomalies_jumpgap)
if "stuck" in selected_detectors:     anomaly_frames.append(df_anomalies_stuck)
if "speed" in selected_detectors:     anomaly_frames.append(df_anomalies_speed)
if "backtrack" in selected_detectors: anomaly_frames.append(df_anomalies_backtrack)
if "repeated" in selected_detectors:  anomaly_frames.append(df_anomalies_repeated)
if "disappear" in selected_detectors: anomaly_frames.append(df_anomalies_disappear)
if "early" in selected_detectors:     anomaly_frames.append(df_anomalies_early)
if "offroute" in selected_detectors:  anomaly_frames.append(df_anomalies_offroute)

anomaly_frames = [df for df in anomaly_frames if 'anomaly_type' in df.columns and not df.empty]
df_anomalies_full = pd.concat(anomaly_frames, ignore_index=True) if anomaly_frames else pd.DataFrame()

# Report
print("Anomaly count (pre-dedup):", len(df_anomalies_full))
df_anomalies_full.drop_duplicates(inplace=True)
print("Anomaly count (post-dedup):", len(df_anomalies_full))

print("✅ Unified anomaly count:", len(df_anomalies_full))
if not df_anomalies_full.empty:
    display(df_anomalies_full["anomaly_type"].value_counts())
else:
    print("⚠️ No anomalies detected.")


Anomaly count (pre-dedup): 161107
Anomaly count (post-dedup): 98785
✅ Unified anomaly count: 98785


,count
anomaly_type,
repeated_points,45903
stuck_vehicle,30035
jump_or_gap,8976
off_route,6766
backtracking,1877
disappearance,1580
impossible_speed,419
early_appearance,1


In [ ]:
df_anomalies_full.describe(include='all')

,vehicle_id,trip_id,timestamp_prev,timestamp_curr,time_diff_sec,distance_m,is_gap,is_jump,anomaly_type,snapshot_id,...,route_color,route_text_color,data_quality,timestamp,computed_speed_kph,heading_prev,heading_diff,last_seen,first_seen,distance_from_route_m
count,95557.000000,95556,14080,9395,8976.000000,9395.000000,8976,8976,95557,77815.000000,...,77815,77815,77815,84581,419.000000,1877.000000,1877.000000,1580,1,6766.000000
unique,NaN,859,NaN,NaN,NaN,NaN,2,2,8,NaN,...,21,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,601256,NaN,NaN,NaN,NaN,False,True,repeated_points,NaN,...,C01D8C,FFFFFF,valid,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,6453,NaN,NaN,NaN,NaN,8965,8972,45903,NaN,...,9657,50766,77815,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,846.262869,NaN,2025-07-23 07:01:24.288907520+00:00,2025-07-23 07:07:24.844485632+00:00,38.012847,666.892952,NaN,NaN,NaN,59.861531,...,NaN,NaN,NaN,2025-07-23 07:00:55.463606272+00:00,134.667901,155.118274,155.548748,2025-07-23 06:42:04.082464512+00:00,2025-07-22 11:17:02.268349952+00:00,1260.436755
min,353.000000,NaN,2025-07-22 11:17:02.268350+00:00,2025-07-22 11:26:37.629258+00:00,30.168835,0.000000,NaN,NaN,NaN,1.000000,...,NaN,NaN,NaN,2025-07-22 11:17:32.471202+00:00,112.722235,0.000000,120.000000,2025-07-22 12:10:26.687556+00:00,2025-07-22 11:17:02.268350+00:00,50.005479
25%,383.000000,NaN,2025-07-22 18:26:52.500549120+00:00,2025-07-22 18:34:33.575398912+00:00,30.210908,540.509796,NaN,NaN,NaN,30.000000,...,NaN,NaN,NaN,2025-07-22 18:35:34.029025024+00:00,116.835730,56.000000,137.000000,2025-07-22 19:02:54.360263680+00:00,2025-07-22 11:17:02.268349952+00:00,90.740586
50%,612.000000,NaN,2025-07-23 12:21:10.761760512+00:00,2025-07-23 12:17:54.381373952+00:00,30.233052,595.932780,NaN,NaN,NaN,60.000000,...,NaN,NaN,NaN,2025-07-23 12:31:00.408647936+00:00,122.156156,162.000000,161.000000,2025-07-23 01:14:53.087767040+00:00,2025-07-22 11:17:02.268349952+00:00,182.807218
75%,959.000000,NaN,2025-07-23 18:54:26.880832512+00:00,2025-07-23 19:08:56.969906944+00:00,30.260672,693.438421,NaN,NaN,NaN,90.000000,...,NaN,NaN,NaN,2025-07-23 19:03:24.204971008+00:00,130.813092,246.000000,173.000000,2025-07-23 18:36:47.726439424+00:00,2025-07-22 11:17:02.268349952+00:00,707.637108
max,4102.000000,NaN,2025-07-24 01:13:00.146041+00:00,2025-07-24 01:13:30.399545+00:00,36085.610497,17251.200687,NaN,NaN,NaN,120.000000,...,NaN,NaN,NaN,2025-07-24 01:13:30.399545+00:00,555.584547,359.000000,180.000000,2025-07-24 01:06:56.592474+00:00,2025-07-22 11:17:02.268350+00:00,19818.882063


In [ ]:
df_anomalies_full.head(100)

,vehicle_id,trip_id,timestamp_prev,timestamp_curr,time_diff_sec,distance_m,is_gap,is_jump,anomaly_type,snapshot_id,...,route_color,route_text_color,data_quality,timestamp,computed_speed_kph,heading_prev,heading_diff,last_seen,first_seen,distance_from_route_m
0,353.0,601002,2025-07-23 12:04:47.943384+00:00,2025-07-23 12:05:18.212398+00:00,30.269014,674.155691,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
1,353.0,601002,2025-07-23 12:07:49.279183+00:00,2025-07-23 12:08:19.471742+00:00,30.192559,704.761635,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
2,353.0,601002,2025-07-23 12:08:49.662742+00:00,2025-07-23 12:09:19.865535+00:00,30.202793,938.014924,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
3,353.0,601002,2025-07-23 12:10:50.476216+00:00,2025-07-23 12:11:20.695650+00:00,30.219434,612.468609,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
4,353.0,601002,2025-07-23 12:11:50.941639+00:00,2025-07-23 12:12:21.142408+00:00,30.200769,606.454757,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,353.0,601013,2025-07-23 17:53:18.816001+00:00,2025-07-23 17:53:49.081738+00:00,30.265737,512.592840,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
96,353.0,601013,2025-07-23 17:55:19.895549+00:00,2025-07-23 17:55:50.068119+00:00,30.172570,648.308913,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
97,353.0,601013,2025-07-23 18:02:22.914930+00:00,2025-07-23 18:02:53.167997+00:00,30.253067,709.891493,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
98,353.0,601013,2025-07-23 18:06:55.430577+00:00,2025-07-23 18:07:25.697310+00:00,30.266733,609.375122,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN


In [ ]:
# print("Total rows:", len(df_anomalies_full))
# print("Unique rows:", len(df_anomalies_full.drop_duplicates()))
# print("Duplicate rows:", df_anomalies_full.duplicated().sum())


Total rows: 161107
Unique rows: 98785
Duplicate rows: 62322


####Merge check
2. Shape-Based Merge into df_anomalies_offroute
- df_anomalies_offroute = df_anomalies_offroute.merge(
    shape_route_map, on="shape_id", how="left", suffixes=("", "_from_map")
)

🔍 Key issue: This assumes that df_anomalies_offroute has an accurate shape_id per row.

But this is not always valid, because:

- Anomalies are grouped by vehicle_id and timestamp.

- There's no explicit trip_id or shape_id preserved per anomaly.

- If a single vehicle changes shapes mid-route, this could assign the wrong route.

In [ ]:
shape_route_map.head()

,route_id,route_short_name
shape_id,,
17128,4300,1
17129,4300,1
17130,4301,10
17131,4301,10
17132,4301,10


In [ ]:
shape_route_map.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67 entries, 17128 to 17151
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   route_id          67 non-null     object
 1   route_short_name  67 non-null     object
dtypes: object(2)
memory usage: 1.6+ KB


In [ ]:
df_anomalies_offroute.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6766 entries, 0 to 6765
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   vehicle_id             6766 non-null   int64              
 1   trip_id                6766 non-null   object             
 2   timestamp              6766 non-null   datetime64[ns, UTC]
 3   route_short_name       6766 non-null   object             
 4   distance_from_route_m  6766 non-null   float64            
 5   latitude               6766 non-null   float64            
 6   longitude              6766 non-null   float64            
 7   shape_id               6766 non-null   object             
 8   anomaly_type           6766 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(3), int64(1), object(4)
memory usage: 475.9+ KB


In [ ]:
df_anomalies_offroute['shape_id'].value_counts()

,count
shape_id,
17143,634
17208,534
17209,451
17212,397
17213,373
17202,347
17211,336
17129,238
17217,231


In [ ]:
# Ensure shape_id is a column (not index)
shape_route_map = shape_route_map.reset_index()

# Ensure both shape_id columns are string type before merge
df_anomalies_offroute["shape_id"] = df_anomalies_offroute["shape_id"].astype(str)
shape_route_map["shape_id"] = shape_route_map["shape_id"].astype(str)

# Join to enrich anomalies with route info
df_anomalies_offroute = (
    df_anomalies_offroute
    .merge(shape_route_map, on="shape_id", how="left", suffixes=("", "_from_map"))
)

# # Patch missing route names
# df_anomalies_offroute["route_short_name"] = (
#     df_anomalies_offroute["route_short_name"]
#     .fillna(df_anomalies_offroute["route_short_name_from_map"])
# )

# # Clean up
# df_anomalies_offroute = df_anomalies_offroute.drop(columns=["route_short_name_from_map"])


In [ ]:
df_anomalies_offroute.describe(include='all')

,vehicle_id,trip_id,timestamp,route_short_name,distance_from_route_m,latitude,longitude,shape_id,anomaly_type,route_id,route_short_name_from_map
count,6766.000000,6766,6766,6766,6766.000000,6766.000000,6766.000000,6766,6766,6766,6766
unique,NaN,563,NaN,22,NaN,NaN,NaN,52,1,22,22
top,NaN,600883,NaN,66,NaN,NaN,NaN,17143,off_route,4328,66
freq,NaN,138,NaN,985,NaN,NaN,NaN,634,6766,985,985
mean,888.149128,NaN,2025-07-23 08:14:48.520649728+00:00,NaN,1260.436755,35.084792,-106.638081,NaN,NaN,NaN,NaN
min,353.000000,NaN,2025-07-22 11:23:35.114951+00:00,NaN,50.005479,34.947920,-106.753930,NaN,NaN,NaN,NaN
25%,459.000000,NaN,2025-07-22 20:10:53.372270080+00:00,NaN,90.740586,35.067865,-106.703012,NaN,NaN,NaN,NaN
50%,614.000000,NaN,2025-07-23 12:39:50.072777984+00:00,NaN,182.807218,35.083990,-106.646795,NaN,NaN,NaN,NaN
75%,970.000000,NaN,2025-07-23 20:50:18.682051072+00:00,NaN,707.637108,35.099986,-106.583750,NaN,NaN,NaN,NaN
max,4102.000000,NaN,2025-07-24 01:13:30.399545+00:00,NaN,19818.882063,35.209220,-106.495960,NaN,NaN,NaN,NaN


In [ ]:
df_anomalies_offroute.isna().sum()

,0
vehicle_id,0
trip_id,0
timestamp,0
route_short_name,0
distance_from_route_m,0
latitude,0
longitude,0
shape_id,0
anomaly_type,0
route_id,0


In [ ]:
#Sanity Check
missing_routes = df_anomalies_offroute["route_short_name"].isnull().sum()
print(f"Remaining anomalies with missing route_short_name: {missing_routes}")
#If this prints 0, you’ve successfully patched all entries.

if missing_routes > 0:
    print("Sample of anomalies with missing route_short_name:")
    display(df_anomalies_offroute[df_anomalies_offroute["route_short_name"].isnull()].head())


Remaining anomalies with missing route_short_name: 0


In [ ]:
df_full.isna().sum()

,0
snapshot_id,0
timestamp_collected,0
vehicle_id,0
latitude,0
longitude,0
heading,0
speed_mph,0
route_short_name,0
trip_id,0
next_stop_id,0


In [ ]:
df_full.columns

Index(['snapshot_id', 'timestamp_collected', 'vehicle_id', 'latitude',
       'longitude', 'heading', 'speed_mph', 'route_short_name', 'trip_id',
       'next_stop_id', 'next_stop_name', 'next_stop_sched_time', 'route_id',
       'service_id', 'trip_headsign', 'trip_short_name', 'direction_id',
       'block_id', 'shape_id', 'wheelchair_accessible', 'bikes_allowed',
       'agency_id', 'route_short_name_static', 'route_long_name', 'route_desc',
       'route_type', 'route_url', 'route_color', 'route_text_color',
       'data_quality'],
      dtype='object')

#####Debugging note: I believe the cell below may be creating some problems

In [ ]:
df_full['data_quality'].value_counts()

,count
data_quality,
valid,171355


In [ ]:
#🔹 1.1 Filter for Valid Rows
df_valid = df_full[df_full["data_quality"] == "valid"].copy()

#🔹 1.2 Parse Timestamps
df_valid["timestamp"] = pd.to_datetime(df_valid["timestamp_collected"], utc=True)

#🔹 1.3 Sort by Vehicle and Timestamp
df_valid = df_valid.sort_values(by=["vehicle_id", "timestamp"])

#🔹 1.4 Organize by Vehicle
#This creates a dictionary keyed by vehicle ID, each with a sorted DataFrame:
vehicle_groups = dict(tuple(df_valid.groupby("vehicle_id")))

#You can confirm how many distinct vehicles you’re tracking:
print("Vehicle count:", len(vehicle_groups))


Vehicle count: 87


#Summary Stats by Route/Anomaly

####Debugging check: possible problem here with route_lookup

In [ ]:
# df_valid.head(20)

,snapshot_id,timestamp_collected,vehicle_id,latitude,longitude,heading,speed_mph,route_short_name,trip_id,next_stop_id,...,agency_id,route_short_name_static,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,data_quality,timestamp
84264,84,2025-07-23 11:57:44.872302,353,35.206084,-106.660050,125.0,9.0,155,601002,704230731,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 11:57:44.872302+00:00
84278,85,2025-07-23 11:58:15.130567,353,35.207074,-106.659887,277.0,17.0,155,601002,704230731,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 11:58:15.130567+00:00
84292,86,2025-07-23 11:58:45.330622,353,35.207163,-106.661616,273.0,1.0,155,601002,704230731,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 11:58:45.330622+00:00
84307,87,2025-07-23 11:59:15.571828,353,35.206109,-106.661820,184.0,0.0,155,601002,704230731,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 11:59:15.571828+00:00
84324,88,2025-07-23 11:59:45.773193,353,35.205764,-106.661329,96.0,23.0,155,601002,704230731,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 11:59:45.773193+00:00
84341,89,2025-07-23 12:00:15.971801,353,35.205642,-106.659603,96.0,31.0,155,601002,704230731,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 12:00:15.971801+00:00
84358,90,2025-07-23 12:00:46.177745,353,35.203312,-106.655098,176.0,30.0,155,601002,704230037,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 12:00:46.177745+00:00
84376,91,2025-07-23 12:01:16.390935,353,35.200974,-106.656233,213.0,10.0,155,601002,704230037,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 12:01:16.390935+00:00
84396,92,2025-07-23 12:01:46.630993,353,35.199446,-106.654031,109.0,30.0,155,601002,704230438,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 12:01:46.630993+00:00
84416,93,2025-07-23 12:02:16.836796,353,35.198117,-106.651578,160.0,12.0,155,601002,1661,...,1,155,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 12:02:16.836796+00:00


In [ ]:
# df_valid.columns

Index(['snapshot_id', 'timestamp_collected', 'vehicle_id', 'latitude',
       'longitude', 'heading', 'speed_mph', 'route_short_name', 'trip_id',
       'next_stop_id', 'next_stop_name', 'next_stop_sched_time', 'route_id',
       'service_id', 'trip_headsign', 'trip_short_name', 'direction_id',
       'block_id', 'shape_id', 'wheelchair_accessible', 'bikes_allowed',
       'agency_id', 'route_short_name_static', 'route_long_name', 'route_desc',
       'route_type', 'route_url', 'route_color', 'route_text_color',
       'data_quality', 'timestamp'],
      dtype='object')

In [ ]:
# routes.columns

Index(['route_id', 'agency_id', 'route_short_name', 'route_long_name',
       'route_desc', 'route_type', 'route_url', 'route_color',
       'route_text_color'],
      dtype='object')

In [ ]:
# trips.columns

Index(['route_id', 'service_id', 'trip_id', 'trip_headsign', 'trip_short_name',
       'direction_id', 'block_id', 'shape_id', 'wheelchair_accessible',
       'bikes_allowed'],
      dtype='object')

In [ ]:
# Build trip-to-route lookup from GTFS static data
trip_to_route_lookup = trips[["trip_id", "route_id"]].merge(
    routes[["route_id", "route_short_name"]], on="route_id", how="left"
)

In [ ]:
trip_to_route_lookup.columns

Index(['trip_id', 'route_id', 'route_short_name'], dtype='object')

In [ ]:
trip_to_route_lookup['route_short_name'].value_counts()

,count
route_short_name,
766,215
777,202
66,198
141,173
11,121
5,110
10,107
8,107
157,105


In [ ]:
trip_to_route_lookup['route_id'].value_counts()

,count
route_id,
4330,215
4331,202
4328,198
4306,173
4340,121
4321,110
4301,107
4333,107
4308,105


In [ ]:
# Build trip-to-route lookup from GTFS static data
trip_to_route_lookup = trips[["trip_id", "route_id"]].merge(
    routes[["route_id", "route_short_name"]], on="route_id", how="left"
)

# Ensure trip_id types match
df_valid["trip_id"] = df_valid["trip_id"].astype(str)
trip_to_route_lookup["trip_id"] = trip_to_route_lookup["trip_id"].astype(str)

#I believe this may have been resolved by splitting route_short_name into x and y
# # Drop route_short_name from df_valid if it exists to avoid merge collision
# if "route_short_name" in df_valid.columns:
#     df_valid = df_valid.drop(columns=["route_short_name"])

trip_to_route_lookup["route_id_trips"] = trip_to_route_lookup["route_id"].copy()
trip_to_route_lookup["route_short_name_trips"] = trip_to_route_lookup["route_short_name"].copy()


# Merge to patch in route_short_name
df_valid = df_valid.merge(trip_to_route_lookup[["trip_id", "route_id_trips", "route_short_name_trips"]], on="trip_id", how="left")


In [ ]:
df_valid.columns

Index(['snapshot_id', 'timestamp_collected', 'vehicle_id', 'latitude',
       'longitude', 'heading', 'speed_mph', 'route_short_name', 'trip_id',
       'next_stop_id', 'next_stop_name', 'next_stop_sched_time', 'route_id',
       'service_id', 'trip_headsign', 'trip_short_name', 'direction_id',
       'block_id', 'shape_id', 'wheelchair_accessible', 'bikes_allowed',
       'agency_id', 'route_short_name_static', 'route_long_name', 'route_desc',
       'route_type', 'route_url', 'route_color', 'route_text_color',
       'data_quality', 'timestamp', 'route_id_trips',
       'route_short_name_trips'],
      dtype='object')

In [ ]:
df_valid.head(10)

,snapshot_id,timestamp_collected,vehicle_id,latitude,longitude,heading,speed_mph,route_short_name,trip_id,next_stop_id,...,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,data_quality,timestamp,route_id_trips,route_short_name_trips
0,84,2025-07-23 11:57:44.872302,353,35.206084,-106.660050,125.0,9.0,155,601002,704230731,...,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 11:57:44.872302+00:00,4307,155
1,85,2025-07-23 11:58:15.130567,353,35.207074,-106.659887,277.0,17.0,155,601002,704230731,...,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 11:58:15.130567+00:00,4307,155
2,86,2025-07-23 11:58:45.330622,353,35.207163,-106.661616,273.0,1.0,155,601002,704230731,...,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 11:58:45.330622+00:00,4307,155
3,87,2025-07-23 11:59:15.571828,353,35.206109,-106.661820,184.0,0.0,155,601002,704230731,...,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 11:59:15.571828+00:00,4307,155
4,88,2025-07-23 11:59:45.773193,353,35.205764,-106.661329,96.0,23.0,155,601002,704230731,...,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 11:59:45.773193+00:00,4307,155
5,89,2025-07-23 12:00:15.971801,353,35.205642,-106.659603,96.0,31.0,155,601002,704230731,...,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 12:00:15.971801+00:00,4307,155
6,90,2025-07-23 12:00:46.177745,353,35.203312,-106.655098,176.0,30.0,155,601002,704230037,...,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 12:00:46.177745+00:00,4307,155
7,91,2025-07-23 12:01:16.390935,353,35.200974,-106.656233,213.0,10.0,155,601002,704230037,...,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 12:01:16.390935+00:00,4307,155
8,92,2025-07-23 12:01:46.630993,353,35.199446,-106.654031,109.0,30.0,155,601002,704230438,...,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 12:01:46.630993+00:00,4307,155
9,93,2025-07-23 12:02:16.836796,353,35.198117,-106.651578,160.0,12.0,155,601002,1661,...,Coors,NaN,3,NaN,FF8900,000000,valid,2025-07-23 12:02:16.836796+00:00,4307,155


In [ ]:
#Step 1: Build a lookup table
route_lookup = df_valid[["vehicle_id", "timestamp", "route_short_name"]].copy()
route_lookup["timestamp"] = pd.to_datetime(route_lookup["timestamp"], utc=True)


In [ ]:
df_anomalies_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98785 entries, 0 to 161106
Data columns (total 44 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   vehicle_id               95557 non-null  float64            
 1   trip_id                  95556 non-null  object             
 2   timestamp_prev           14080 non-null  datetime64[ns, UTC]
 3   timestamp_curr           9395 non-null   datetime64[ns, UTC]
 4   time_diff_sec            8976 non-null   float64            
 5   distance_m               9395 non-null   float64            
 6   is_gap                   8976 non-null   object             
 7   is_jump                  8976 non-null   object             
 8   anomaly_type             95557 non-null  object             
 9   snapshot_id              77815 non-null  float64            
 10  timestamp_collected      77815 non-null  datetime64[ns]     
 11  latitude                 84581 n

In [ ]:
#I believe this is already resolved
# #Step 2: Also convert timestamp in anomalies to datetime
# df_anomalies_full["timestamp"] = pd.to_datetime(df_anomalies_full["timestamp"], utc=True)


In [ ]:
#Create a backup so we don't have to keep rerunning everything
df_anomalies_full_copy = df_anomalies_full.copy()

# #Rebuild from scratch:

# df_anomalies_full_backup = pd.concat([
    # df_anomalies_jumpgap,
    # df_anomalies_stuck,
    # df_anomalies_speed,
    # df_anomalies_backtrack,
    # df_anomalies_repeated,
    # df_anomalies_disappear,
    # df_anomalies_early,
    # df_anomalies_offroute
# ], ignore_index=True)

In [ ]:
# df_anomalies_full = df_anomalies_full_copy

✅ df_anomalies_full_copy has the correct anomaly types:
repeated_points     46200  
stuck_vehicle       30988  
jump_or_gap          9200  ← LOST  
off_route            6767  
backtracking         1616  
impossible_speed      434  ← LOST  
disappearance          42  ← LOST  
early_appearance        1  ← LOST  
But after the route name merge, your current df_anomalies_full is missing the last four types.


####Merge check
(Note: this was resolved in .v8)
3. merge_asof Join Between df_anomalies_full and route_lookup
- merge_result = pd.merge_asof(
    df_anomalies_full.sort_values("timestamp"),
    route_lookup.sort_values("timestamp"),
    on="timestamp",
    by="vehicle_id",
    direction="nearest",
    ...
)

🔍 Potential source of corruption:
- If route_lookup (used to get route IDs for vehicles) was built without tight trip_id / shape_id constraints, it can erroneously associate the wrong route to a vehicle.
- merge_asof can assign the nearest route in time — but this may be a different shape or trip.

🛑 This is a prime suspect for inflated off_route counts.

In [ ]:
#Part of the merge investigation; can be removed later
# # Check how many unique routes are assigned per vehicle
# route_counts = df_anomalies_full.groupby("vehicle_id")["route_short_name"].nunique()
# conflict_vehicles = route_counts[route_counts > 1].index

# print(f"🚨 Vehicles with conflicting routes: {len(conflict_vehicles)}")


🚨 Vehicles with conflicting routes: 0


In [ ]:
# #Diagnostics Step 1: See how many non-null entries are already in merge_subset["route_short_name"]
# print("🔎 Pre-merge route_short_name stats in merge_subset:")
# print(merge_subset["route_short_name"].value_counts(dropna=False).head(10))
# print("Total non-null route_short_name:", merge_subset["route_short_name"].notna().sum())


🔎 Pre-merge route_short_name stats in merge_subset:
route_short_name
NaN     78804
None     6767
Name: count, dtype: int64
Total non-null route_short_name: 0


In [ ]:
# #Diagnostics Step 2: Perform the exact merge but using a temporary suffix to compare side-by-side
# merged_temp = pd.merge(
#     merge_subset,
#     route_lookup_clean[["vehicle_id", "timestamp", "route_short_name"]],
#     on=["vehicle_id", "timestamp"],
#     how="left",
#     suffixes=("_original", "_lookup")
# )


In [ ]:
# #Diagnostics Step 3: Check how often the two columns agree, disagree, or one is missing
# both_present = merged_temp["route_short_name_original"].notna() & merged_temp["route_short_name_lookup"].notna()
# same = merged_temp["route_short_name_original"] == merged_temp["route_short_name_lookup"]
# disagree = both_present & ~same

# print("✅ Both present:", both_present.sum())
# print("✅ Agreeing values:", (same & both_present).sum())
# print("❌ Disagreeing values:", disagree.sum())


✅ Both present: 0
✅ Agreeing values: 0
❌ Disagreeing values: 0


In [ ]:
# #Diagnostics Step 4: View some of the disagreements
# print("⚠️ Sample rows where route_short_name disagrees:")
# display(merged_temp.loc[disagree, ["vehicle_id", "timestamp", "route_short_name_original", "route_short_name_lookup"]].head(10))


⚠️ Sample rows where route_short_name disagrees:


,vehicle_id,timestamp,route_short_name_original,route_short_name_lookup


In [ ]:
# #Diagnostics Step 5: Inspect which source is more complete
# print("Original non-null:", merged_temp["route_short_name_original"].notna().sum())
# print("Lookup non-null:", merged_temp["route_short_name_lookup"].notna().sum())


Original non-null: 0
Lookup non-null: 86960


In [ ]:
df_anomalies_full.head()

,vehicle_id,trip_id,timestamp_prev,timestamp_curr,time_diff_sec,distance_m,is_gap,is_jump,anomaly_type,snapshot_id,...,route_color,route_text_color,data_quality,timestamp,computed_speed_kph,heading_prev,heading_diff,last_seen,first_seen,distance_from_route_m
0,353.0,601002,2025-07-23 12:04:47.943384+00:00,2025-07-23 12:05:18.212398+00:00,30.269014,674.155691,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
1,353.0,601002,2025-07-23 12:07:49.279183+00:00,2025-07-23 12:08:19.471742+00:00,30.192559,704.761635,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
2,353.0,601002,2025-07-23 12:08:49.662742+00:00,2025-07-23 12:09:19.865535+00:00,30.202793,938.014924,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
3,353.0,601002,2025-07-23 12:10:50.476216+00:00,2025-07-23 12:11:20.695650+00:00,30.219434,612.468609,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
4,353.0,601002,2025-07-23 12:11:50.941639+00:00,2025-07-23 12:12:21.142408+00:00,30.200769,606.454757,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN


In [ ]:
#Let's redo the merging *.v8.2
#Actually... not sure why we were doing this at all
#This is from *.v8 addition, basically imposing exact matches on the merge, opposed to an earlier version that would use merge_asof and perform nearest matching
# 🧼 Step 1: Clean merge inputs
valid_rows = df_anomalies_full["timestamp"].notna()
merge_subset = df_anomalies_full.loc[valid_rows].copy()
route_lookup_clean = route_lookup[route_lookup["timestamp"].notna()].copy()

# 🔧 Ensure matching dtypes
merge_subset["vehicle_id"] = merge_subset["vehicle_id"].astype(str)
route_lookup_clean["vehicle_id"] = route_lookup_clean["vehicle_id"].astype(str)

In [ ]:
merge_subset.columns

Index(['vehicle_id', 'trip_id', 'timestamp_prev', 'timestamp_curr',
       'time_diff_sec', 'distance_m', 'is_gap', 'is_jump', 'anomaly_type',
       'snapshot_id', 'timestamp_collected', 'latitude', 'longitude',
       'heading', 'speed_mph', 'route_short_name', 'next_stop_id',
       'next_stop_name', 'next_stop_sched_time', 'route_id', 'service_id',
       'trip_headsign', 'trip_short_name', 'direction_id', 'block_id',
       'shape_id', 'wheelchair_accessible', 'bikes_allowed', 'agency_id',
       'route_short_name_static', 'route_long_name', 'route_desc',
       'route_type', 'route_url', 'route_color', 'route_text_color',
       'data_quality', 'timestamp', 'computed_speed_kph', 'heading_prev',
       'heading_diff', 'last_seen', 'first_seen', 'distance_from_route_m'],
      dtype='object')

In [ ]:
route_lookup_clean.columns

Index(['vehicle_id', 'timestamp', 'route_short_name'], dtype='object')

In [ ]:
# merge_redux = merge_subset.merge(route_lookup_clean[['vehicle_id', 'timestamp']])

In [ ]:
# #This is the *.v8 addition, basically imposing exact matches on the merge, opposed to an earlier version that would use merge_asof and perform nearest matching
# # 🧼 Step 1: Clean merge inputs
# valid_rows = df_anomalies_full["timestamp"].notna()
# merge_subset = df_anomalies_full.loc[valid_rows].copy()
# route_lookup_clean = route_lookup[route_lookup["timestamp"].notna()].copy()

# # 🔧 Ensure matching dtypes
# merge_subset["vehicle_id"] = merge_subset["vehicle_id"].astype(str)
# route_lookup_clean["vehicle_id"] = route_lookup_clean["vehicle_id"].astype(str)

# # ✅ Drop empty original route column
# merge_subset = merge_subset.drop(columns=["route_short_name"], errors="ignore")

# # 🔁 Step 2: Exact merge
# merged = pd.merge(
#     merge_subset,
#     route_lookup_clean[["vehicle_id", "timestamp", "route_short_name"]],
#     on=["vehicle_id", "timestamp"],
#     how="left"
# )

# # 🔁 Step 3: Patch back into df_anomalies_full
# route_name_map = merged.set_index(merge_subset.index)["route_short_name"]
# df_anomalies_full["route_short_name"] = df_anomalies_full["route_short_name"].combine_first(route_name_map)

# # ✅ Step 4: Final check
# print("✅ Remaining missing route names:", df_anomalies_full["route_short_name"].isna().sum())


✅ Remaining missing route names: 98785


#####*.v8.2 250731-1630 note: not sure why we were cleaning the data above
and I think this merging may have caused some issues, so skipping it for now

In [ ]:
df_anomalies_full[df_anomalies_full["route_short_name"].isna()].sample(5)


,vehicle_id,trip_id,timestamp_prev,timestamp_curr,time_diff_sec,distance_m,is_gap,is_jump,anomaly_type,snapshot_id,...,route_color,route_text_color,data_quality,timestamp,computed_speed_kph,heading_prev,heading_diff,last_seen,first_seen,distance_from_route_m
6790,971.0,600316,2025-07-22 20:32:34.541360+00:00,2025-07-22 20:33:04.755652+00:00,30.214292,618.577900,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
3700,603.0,601156,2025-07-23 14:40:07.005075+00:00,2025-07-23 14:40:37.288022+00:00,30.282947,626.850682,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
7564,1904.0,602970,2025-07-23 12:08:49.662742+00:00,2025-07-23 12:09:19.865535+00:00,30.202793,535.555859,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
644,359.0,601176,2025-07-23 15:01:17.888676+00:00,2025-07-23 15:01:48.096383+00:00,30.207707,525.849678,False,True,jump_or_gap,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaN
153854,952.0,601594,NaT,NaT,NaN,NaN,NaN,NaN,disappearance,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,2025-07-23 20:38:00.926406+00:00,NaT,NaN


In [ ]:
df_anomalies_full.isna().sum()

,0
vehicle_id,3228
trip_id,3229
timestamp_prev,84705
timestamp_curr,89390
time_diff_sec,89809
distance_m,89390
is_gap,89809
is_jump,89809
anomaly_type,3228
snapshot_id,20970


##Debugging waypoint 2507251634

In [ ]:
df_anomalies_full['route_short_name'].value_counts()

,count
route_short_name,
16,9991
66,7851
766,7110
141,6866
777,5702
8,4888
5,4787
157,3982
155,3765


In [ ]:
#Presently there are no unknowns or nulls, so we don't have to worry about this
# # Treat 'Unknown' as null
# df_anomalies_full["route_short_name"] = df_anomalies_full["route_short_name"].replace("Unknown", pd.NA)

# # Only apply fallback if the lookup column is still present
# if "route_short_name_from_lookup" in df_anomalies_full.columns:
#     df_anomalies_full["route_short_name"] = df_anomalies_full["route_short_name"].combine_first(
#         df_anomalies_full["route_short_name_from_lookup"]
#     )

# # Confirm
# print("Remaining missing route names:", df_anomalies_full["route_short_name"].isna().sum())
# print(df_anomalies_full["route_short_name"].value_counts(dropna=False).head())


Remaining missing route names: 98785
route_short_name
NaN    98785
Name: count, dtype: int64


In [ ]:
#Same as above cell
# # Confirm null replacement and summary
# df_anomalies_full["route_short_name"] = df_anomalies_full["route_short_name"].replace("Unknown", pd.NA)

# # Summary diagnostics
# print("Remaining missing route names:", df_anomalies_full["route_short_name"].isna().sum())
# print(df_anomalies_full["route_short_name"].value_counts(dropna=False).head())


In [ ]:
print("Checkpoint anomaly type breakdown:")
print(df_anomalies_full["anomaly_type"].value_counts(dropna=False))

print("Checkpoint anomaly type breakdown (copy):")
print(df_anomalies_full_copy["anomaly_type"].value_counts(dropna=False))

Checkpoint anomaly type breakdown:
anomaly_type
repeated_points     45903
stuck_vehicle       30035
jump_or_gap          8976
off_route            6766
NaN                  3228
backtracking         1877
disappearance        1580
impossible_speed      419
early_appearance        1
Name: count, dtype: int64
Checkpoint anomaly type breakdown (copy):
anomaly_type
repeated_points     45903
stuck_vehicle       30035
jump_or_gap          8976
off_route            6766
NaN                  3228
backtracking         1877
disappearance        1580
impossible_speed      419
early_appearance        1
Name: count, dtype: int64


In [ ]:
print("Unique types in anomaly_type:")
print(set(type(val) for val in df_anomalies_full["anomaly_type"].unique()))


Unique types in anomaly_type:
{<class 'float'>, <class 'str'>}


In [ ]:
print("Full anomaly types (raw):", df_anomalies_full["anomaly_type"].unique())
print("Full anomaly types (with types):", [(val, type(val)) for val in df_anomalies_full["anomaly_type"].unique()])


Full anomaly types (raw): ['jump_or_gap' 'stuck_vehicle' 'impossible_speed' 'backtracking' nan
 'repeated_points' 'disappearance' 'early_appearance' 'off_route']
Full anomaly types (with types): [('jump_or_gap', <class 'str'>), ('stuck_vehicle', <class 'str'>), ('impossible_speed', <class 'str'>), ('backtracking', <class 'str'>), (nan, <class 'float'>), ('repeated_points', <class 'str'>), ('disappearance', <class 'str'>), ('early_appearance', <class 'str'>), ('off_route', <class 'str'>)]


In [ ]:
print(selected_detectors)

['jumpgap', 'stuck', 'speed', 'backtrack', 'repeated', 'disappear', 'early', 'offroute']


In [ ]:
df_anomalies_full["anomaly_type"].unique()

array(['jump_or_gap', 'stuck_vehicle', 'impossible_speed', 'backtracking',
       nan, 'repeated_points', 'disappearance', 'early_appearance',
       'off_route'], dtype=object)

In [ ]:
# Step 1: Clean anomaly type column
df_temp = df_anomalies_full.copy()
df_temp["route_short_name"] = df_temp["route_short_name"].fillna("(Missing)")
df_temp["anomaly_type_clean"] = df_temp["anomaly_type"].astype(str)

# Step 2: Build pivot table
pivot = df_temp.pivot_table(
    index="route_short_name",
    columns="anomaly_type_clean",
    aggfunc="size",
    fill_value=0
)

# Step 3: Manually reindex using the full list
expected_columns = [
    "repeated_points", "stuck_vehicle", "jump_or_gap", "off_route",
    "backtracking", "impossible_speed", "disappearance", "early_appearance"
]
pivot = pivot.reindex(columns=expected_columns, fill_value=0)

# Step 4: Diagnostics
print("✅ Total in pivot:", pivot.sum().sum())
print("✅ Total in df:", df_anomalies_full["anomaly_type"].notna().sum())
print("✅ Columns in pivot:", pivot.columns.tolist())


✅ Total in pivot: 95557
✅ Total in df: 95557
✅ Columns in pivot: ['repeated_points', 'stuck_vehicle', 'jump_or_gap', 'off_route', 'backtracking', 'impossible_speed', 'disappearance', 'early_appearance']


Key Fix Summary (for documentation or future review):
- Problem cause: pivot_table(...).reindex(columns=sorted(...)) introduced TypeError due to sorting mixed types (possibly hidden NaNs or object-type weirdness in internal index representation).

- Confirmed all anomaly_type values were str, but reindexing still failed — indicating a more subtle index-level mismatch.

Solution: Explicitly cast anomaly_type to string and reindex with a predefined list of expected columns.

In [ ]:
pivot

anomaly_type_clean,repeated_points,stuck_vehicle,jump_or_gap,off_route,backtracking,impossible_speed,disappearance,early_appearance
route_short_name,,,,,,,,
(Missing),0,0,8976,0,0,419,1580,1
1,1646,1241,0,333,37,0,0,0
10,1828,1092,0,100,103,0,0,0
11,1922,1232,0,104,49,0,0,0
140,1199,829,0,118,26,0,0,0
141,3757,2398,0,628,83,0,0,0
155,2105,1211,0,276,173,0,0,0
157,2271,1289,0,276,146,0,0,0
16,5132,4436,0,334,89,0,0,0


In [ ]:
#Second pivot table to try looking at things other than route_short_name
# Step 1: Clean anomaly type column
df_temp2 = df_anomalies_full.copy()
df_temp2["trip_id"] = df_temp2["trip_id"].fillna("(Missing)")
df_temp2["anomaly_type_clean"] = df_temp2["anomaly_type"].astype(str)

# Step 2: Build pivot table
pivot2 = df_temp2.pivot_table(
    index="trip_id",
    columns="anomaly_type_clean",
    aggfunc="size",
    fill_value=0
)

# Step 3: Manually reindex using the full list
expected_columns = [
    "repeated_points", "stuck_vehicle", "jump_or_gap", "off_route",
    "backtracking", "impossible_speed", "disappearance", "early_appearance"
]
pivot2 = pivot2.reindex(columns=expected_columns, fill_value=0)

# Step 4: Diagnostics
print("✅ Total in pivot:", pivot2.sum().sum())
print("✅ Total in df:", df_anomalies_full["anomaly_type"].notna().sum())
print("✅ Columns in pivot:", pivot2.columns.tolist())

✅ Total in pivot: 95557
✅ Total in df: 95557
✅ Columns in pivot: ['repeated_points', 'stuck_vehicle', 'jump_or_gap', 'off_route', 'backtracking', 'impossible_speed', 'disappearance', 'early_appearance']


In [ ]:
pivot2

anomaly_type_clean,repeated_points,stuck_vehicle,jump_or_gap,off_route,backtracking,impossible_speed,disappearance,early_appearance
trip_id,,,,,,,,
(Missing),0,0,0,0,0,0,0,1
600305,37,30,15,1,2,0,2,0
600306,37,27,14,2,1,0,2,0
600307,70,52,17,1,2,3,2,0
600308,63,56,11,0,3,0,2,0
...,...,...,...,...,...,...,...,...
603198,19,5,5,18,3,0,2,0
603199,10,4,4,9,3,0,2,0
603200,10,2,2,8,1,0,2,0


####Debuggin in *.v8, to do:
Thanks for the clear context and diagnostic output — this is an important signal, and you're absolutely right to question it.

---

### 🔍 What's Going Wrong

The anomaly types:

* `"jump_or_gap"`, `"impossible_speed"`, `"disappearance"`, `"early_appearance"`

are **only appearing under `(Missing)` routes**, while all known route numbers (like `1`, `16`, `66`) show `0` in those columns.

Yet we know these anomaly types **should be assigned to real routes** in many cases.

---

### 🧠 Root Cause

Those anomalies **likely didn't get their `route_short_name` correctly assigned** during the earlier merge. Here's why:

* Many of these anomaly types are generated from **timestamp-level snapshot comparisons**, often between vehicles’ **first or last records**, or **with subtle time gaps**.
* If the timestamp didn’t match exactly with a route in the `route_lookup`, and you removed `merge_asof`, they **weren’t matched at all**.
* So they fell through with missing `route_short_name`, and got lumped into the `"(Missing)"` category in the pivot.

In short:

> You now get only **exact route matches**, which means **no match = Missing = misattributed anomalies**.

---

### ✅ Recommended Fix

To preserve strict merge logic *but still capture route info for jump/disappearance-type anomalies*, you need a **separate merge strategy** for those rows. Here's how:

---

#### 🔄 Updated Strategy

1. **Retain your current exact-match logic** for most anomalies (good for accuracy).
2. **For rows that still have no route**, use a **second fallback join**:

   * Join on `(vehicle_id, rounded timestamp)` with small ± tolerance via *manual logic*, **not `merge_asof`**.
   * Or pre-fill route for vehicles known to follow the same route across time.

Alternatively, if you're okay adding a **hybrid merge strategy**, then:

* First: `merge` (strict)
* Then: `merge_asof` only on rows that are still `NaN` for route
* Flag those rows as `fallback_match = True` for auditability

---

### 🚧 Immediate Diagnostic You Can Try

Let’s confirm our theory first.

Run this:

```python
missing_routes_df = df_anomalies_full[df_anomalies_full["route_short_name"].isna()]
print("🚫 Missing route entries by anomaly type:")
print(missing_routes_df["anomaly_type"].value_counts())
```

You’ll probably see that all the `jump_or_gap`, `impossible_speed`, etc. live in this `missing_routes_df`.

---

### 🛠️ Optional Patch to Recover These

If you’re willing to use a **limited scope fallback merge**, try this:

```python
# Find rows that still have missing route
missing_route_rows = df_anomalies_full["route_short_name"].isna()

# Attempt fallback merge on these using merge_asof (only here)
fallback_merge = pd.merge_asof(
    df_anomalies_full.loc[missing_route_rows].sort_values("timestamp"),
    route_lookup_clean.sort_values("timestamp"),
    on="timestamp",
    by="vehicle_id",
    direction="nearest",
    tolerance=pd.Timedelta("60s")
)

# Patch only where it's still missing
df_anomalies_full.loc[missing_route_rows, "route_short_name"] = fallback_merge["route_short_name"]
```

⚠️ This preserves your integrity for the main dataset, and **only relaxes the constraint for those otherwise unclassified rows**.

---

Would you like help implementing that fallback merge and visual diagnostic?


** Note: this was debugging notes from ≤*.v6 **

Interestingly, the (Missing) route_short_name has the only instances of jump_or_gap, impossible_speed, disappearance, early_appearance


That’s a critical diagnostic insight — and it likely explains why those anomaly types kept disappearing in earlier pivot tables:

🔍 Root Cause:
By default, pivot_table uses only the combination of index and columns values that exist. If all rows for some anomaly types are associated with a missing route_short_name (NaN), and you drop or exclude these during preprocessing, those types vanish from the pivot.

When you did:

df["route_short_name"] = df["route_short_name"].fillna("(Missing)")

You ensured that:

Those rows were not excluded from the index.

Their anomaly types were included and preserved in the pivot.

✅ Implications:
You must preserve (Missing) rows if you want a complete count across all anomaly types.

If you filter by route, or use .dropna() on route_short_name, anomaly types with only missing routes will be silently excluded.

When you view a pivot per route, types like jump_or_gap, impossible_speed, and disappearance may seem to “disappear” — but they’re actually only found in the (Missing) bucket.

---
To ensure full awareness in reporting or visualizing these anomaly types, do one of the following:

Always include a (Missing) category in route-level pivots.

Add a diagnostic count like:


In [ ]:
print("Anomalies with missing route names by type:")
print(df_anomalies_full[df_anomalies_full["route_short_name"].isna()]["anomaly_type"].value_counts())


Anomalies with missing route names by type:
anomaly_type
jump_or_gap         8976
disappearance       1580
impossible_speed     419
early_appearance       1
Name: count, dtype: int64


In [ ]:
df_anomalies_full["anomaly_type"].value_counts(dropna=False)


,count
anomaly_type,
repeated_points,45903
stuck_vehicle,30035
jump_or_gap,8976
off_route,6766
NaN,3228
backtracking,1877
disappearance,1580
impossible_speed,419
early_appearance,1


#Plot in Folium

In [ ]:
#I don't think this is actually useful at this point
# # Step 1: Create a working copy
# df_plot_base = df_anomalies_full.copy()

# # Step 2: Drop rows with missing coordinates or timestamps
# df_plot_base = df_plot_base.dropna(subset=["latitude", "longitude", "timestamp_collected"])

# # Step 3: Ensure coordinate and timestamp types are correct
# df_plot_base["latitude"] = pd.to_numeric(df_plot_base["latitude"], errors="coerce")
# df_plot_base["longitude"] = pd.to_numeric(df_plot_base["longitude"], errors="coerce")
# df_plot_base["timestamp_collected"] = pd.to_datetime(df_plot_base["timestamp_collected"], errors="coerce", utc=True)

# # Step 4: Optional — Filter for anomalies with known route_short_name
# df_plot_base = df_plot_base[df_plot_base["route_short_name"].notna()]

# # Step 5: Optional — Save a filtered subset by anomaly type, route, or vehicle for visualization
# # You can update these filters dynamically for an interactive tool
# selected_anomaly_type = "stuck_vehicle"
# selected_route = "66"
# # Example: match on anomaly type and route
# df_plot_filtered = df_plot_base[
#     (df_plot_base["anomaly_type"] == selected_anomaly_type) &
#     (df_plot_base["route_short_name"] == selected_route)
# ]

# # Preview result
# print(f"🗺️ Plotting subset for anomaly: '{selected_anomaly_type}' on route '{selected_route}'")
# print(df_plot_filtered[["vehicle_id", "timestamp_collected", "latitude", "longitude", "anomaly_type"]].head())


🗺️ Plotting subset for anomaly: 'stuck_vehicle' on route '66'
Empty DataFrame
Columns: [vehicle_id, timestamp_collected, latitude, longitude, anomaly_type]
Index: []


In [ ]:
# Use full anomaly dataframe
df_full2 = df_anomalies_full.copy()

# Ensure lat/lon are floats
df_full2["latitude"] = pd.to_numeric(df_full2["latitude"], errors="coerce")
df_full2["longitude"] = pd.to_numeric(df_full2["longitude"], errors="coerce")

# Ensure timestamp_collected is datetime
df_full2["timestamp_collected"] = pd.to_datetime(df_full2["timestamp_collected"], errors="coerce", utc=True)

# Compute diffs for position and time
df_full2["lat_diff"] = df_full2.groupby("vehicle_id")["latitude"].diff()
df_full2["lon_diff"] = df_full2.groupby("vehicle_id")["longitude"].diff()
df_full2["jump_dist"] = (df_full2["lat_diff"]**2 + df_full2["lon_diff"]**2)**0.5
df_full2["time_diff"] = df_full2.groupby("vehicle_id")["timestamp_collected"].diff().dt.total_seconds()

# Label jump and disappearance events using anomaly_params
df_full2["is_jump"] = df_full2["jump_dist"] > anomaly_params["JUMP_DISTANCE_THRESHOLD"]
df_full2["is_disappearance"] = df_full2["time_diff"] > anomaly_params["DISAPPEARANCE_TIME_THRESHOLD"]

# Create enriched jump DataFrame
jumps_df = df_full2[df_full2["is_jump"]].copy()
jumps_df["lat_prev"] = df_full2.groupby("vehicle_id")["latitude"].shift()
jumps_df["lon_prev"] = df_full2.groupby("vehicle_id")["longitude"].shift()
jumps_df["timestamp_prev"] = df_full2.groupby("vehicle_id")["timestamp_collected"].shift()
jumps_df["timestamp_curr"] = jumps_df["timestamp_collected"]

# Filter to vehicles with enough jumps
jump_counts = jumps_df["vehicle_id"].value_counts()
keep_jumpers = jump_counts[jump_counts >= anomaly_params["MIN_JUMP_COUNT_PER_VEHICLE"]].index
jumps_df = jumps_df[jumps_df["vehicle_id"].isin(keep_jumpers)]

# Base frame of jumpers for disappearance/reappearance
df_jumpers_only = df_full2[df_full2["vehicle_id"].isin(keep_jumpers)].copy()

# Identify disappearance and reappearance points
disappear_df = df_jumpers_only[df_jumpers_only["is_disappearance"]].copy()
is_reappear = df_jumpers_only["is_disappearance"].shift(-1).fillna(False)
reappear_df = df_jumpers_only[is_reappear].copy()

# Summary of Key Changes:
# ✅ Replaces JUMP_DISTANCE_THRESHOLD, DISAPPEARANCE_TIME_THRESHOLD, and MIN_JUMP_COUNT_PER_VEHICLE with their anomaly_params equivalents.
# ✅ Maintains existing structure and logic.
# ✅ Compatible with all downstream Folium plotting layers.


/tmp/ipython-input-794306693.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_reappear = df_jumpers_only["is_disappearance"].shift(-1).fillna(False)


In [ ]:
#I believe this can be depricated
# # that traceback confirms the immediate issue: after merging into shapes_df, the column route_short_name is not present, likely due to:

# # A failed merge (i.e. no matching shape_id values across shapes_df and shape_route_map).

# # The column being named something else (e.g. a merge conflict that renamed it, or a typo).

# # An earlier shapes_df overwrite or clean-up step that removed key columns.

# print("🔍 Columns in shapes_df:", shapes_df.columns.tolist())

# # Check if shape_route_map exists and contains route_short_name
# if 'shape_route_map' in globals():
#     print("✅ Columns in shape_route_map:", shape_route_map.columns.tolist())
#     print("✅ Sample shape_route_map rows:")
#     display(shape_route_map.head())
# else:
#     print("❌ shape_route_map does not exist")

# # Check shape_id overlap
# if 'shapes_df' in globals():
#     print("🧪 shape_id in shapes_df:", shapes_df["shape_id"].nunique())
#     print("🧪 shape_id in shape_route_map:", shape_route_map["shape_id"].nunique())
#     print("🧪 Overlapping shape_ids:", len(set(shapes_df["shape_id"]).intersection(shape_route_map["shape_id"])))


🔍 Columns in shapes_df: ['shape_id', 'shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence', 'shape_dist_traveled']
✅ Columns in shape_route_map: ['shape_id', 'route_id', 'route_short_name']
✅ Sample shape_route_map rows:


,shape_id,route_id,route_short_name
0,17128,4300,1
1,17129,4300,1
2,17130,4301,10
3,17131,4301,10
4,17132,4301,10


🧪 shape_id in shapes_df: 67
🧪 shape_id in shape_route_map: 67
🧪 Overlapping shape_ids: 0


In [ ]:
#Step 1: Extract shape_id to route_id from trips_df
shape_to_route = trips[["shape_id", "route_id"]].drop_duplicates()
# trips.txt defines which route uses which shape.
# shape_to_route gives a clean 1:1 or 1:many mapping.

#Step 2: Map route_id to route_short_name from routes_df
route_id_to_name = routes[["route_id", "route_short_name"]]
#Brings in human-readable route names like "1", "10", "66".

#Step 3: Merge to associate shape_id with route_short_name
shape_route_map = shape_to_route.merge(route_id_to_name, on="route_id", how="left")
#You now have a table with: shape_id, route_id, and route_short_name

shapes_copy = shapes.copy()

# Step 4: Drop preexisting route_short_name to avoid merge conflicts
if "route_short_name" in shapes_copy.columns:
    shapes_copy.drop(columns=["route_short_name"], inplace=True)

# Step 4 (retry): Merge into shapes_df
shapes_df = shapes_copy.merge(shape_route_map, on="shape_id", how="left")

# Step 5: Confirm structure
assert shapes_df["route_short_name"].notna().all(), "❌ Some shapes are missing route names!"
print("✅ Final columns in shapes_df:", shapes_df.columns.tolist())
display(shapes_df[["shape_id", "route_short_name"]].drop_duplicates().head())





✅ Final columns in shapes_df: ['shape_id', 'shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence', 'shape_dist_traveled', 'route_id', 'route_short_name']


,shape_id,route_short_name
0,17128,1
251,17129,1
441,17130,10
608,17131,10
685,17132,10


In [ ]:
#I believe this can be depricated
# # Next Step: Clean up unnecessary suffix columns
# # Since the final route_short_name column is already correct, and we don’t need the duplicated *_x and *_y columns, you can safely drop them:
# # Clean up leftover duplicate columns
# cols_to_drop = [col for col in shapes_df.columns if col.endswith("_x") or col.endswith("_y")]
# shapes_df.drop(columns=cols_to_drop, inplace=True)

# # Final confirmation
# print("✅ Cleaned columns in shapes_df:", shapes_df.columns.tolist())
# display(shapes_df[["shape_id", "route_short_name"]].drop_duplicates().head())


✅ Cleaned columns in shapes_df: ['shape_id', 'shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence', 'shape_dist_traveled', 'route_id', 'route_short_name']


,shape_id,route_short_name
0,17128,1
251,17129,1
441,17130,10
608,17131,10
685,17132,10


Summary of Current shapes_df Columns:
- shape_id: Unique shape identifier
- shape_pt_lat, shape_pt_lon: GPS points to trace the route shape
- shape_pt_sequence: Order of points to draw the shape
- shape_dist_traveled: Optional, often used to interpolate positions
- route_id, route_short_name: Now both included — route_short_name is the human-readable one you’ll want for plotting and filtering

In [ ]:
# --- Create base map ---
mymap = Map(location=[35.0844, -106.6504], zoom_start=12)

# --- Define color map for routes ---
route_names = sorted(df_anomalies_full['route_short_name'].dropna().unique())
cmap = plt.get_cmap("tab20", len(route_names))
color_map = {route: mcolors.to_hex(cmap(i)) for i, route in enumerate(route_names)}

# --- Plot anomaly markers by route and type ---
for (route, anomaly), subset in df_anomalies_full.groupby(["route_short_name", "anomaly_type"]):
    group = FeatureGroup(name=f"{route} – {anomaly}", show=False)
    for _, row in subset.iterrows():
        if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
            popup = (
                f"Anomaly: {anomaly}<br>"
                f"Route: {route}<br>"
                f"Vehicle: {row.get('vehicle_id', 'N/A')}<br>"
                f"Timestamp: {row.get('timestamp_collected', 'N/A')}"
            )
            CircleMarker(
                location=[row["latitude"], row["longitude"]],
                radius=4,
                color=color_map.get(route, "black"),
                fill=True,
                fill_opacity=0.9,
                popup=popup
            ).add_to(group)
    group.add_to(mymap)

# --- Plot jump lines and markers ---
if 'jumps_df' in globals() and not jumps_df.empty:
    for route, group_df in jumps_df.groupby("route_short_name"):
        jump_line_group = FeatureGroup(name=f"{route} – Jump Lines", show=False)
        jump_point_group = FeatureGroup(name=f"{route} – Jump Start/End", show=False)
        for _, row in group_df.iterrows():
            if all(pd.notna([row["lat_prev"], row["lon_prev"], row["latitude"], row["longitude"]])):
                start = [row["lat_prev"], row["lon_prev"]]
                end = [row["latitude"], row["longitude"]]
                vehicle = row.get("vehicle_id", "N/A")
                t_prev = row.get("timestamp_prev", "N/A")
                t_curr = row.get("timestamp_curr", "N/A")

                PolyLine([start, end], color="orange", weight=2,
                         tooltip=f"Vehicle {vehicle} jump").add_to(jump_line_group)

                CircleMarker(location=start, radius=4, color="blue", fill=True,
                             fill_opacity=0.9, popup=f"START – {vehicle}<br>{t_prev}").add_to(jump_point_group)
                CircleMarker(location=end, radius=4, color="purple", fill=True,
                             fill_opacity=0.9, popup=f"END – {vehicle}<br>{t_curr}").add_to(jump_point_group)
        jump_line_group.add_to(mymap)
        jump_point_group.add_to(mymap)

# --- Plot disappearances ---
if 'disappear_df' in globals() and not disappear_df.empty:
    for route, group_df in disappear_df.groupby("route_short_name"):
        disappear_group = FeatureGroup(name=f"{route} – Disappearances", show=False)
        for _, row in group_df.iterrows():
            if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
                Marker(
                    location=[row["latitude"], row["longitude"]],
                    icon=Icon(color="red", icon="times-circle", prefix="fa"),
                    tooltip=f"Vehicle {row.get('vehicle_id')} disappeared<br>{row.get('timestamp_collected')}"
                ).add_to(disappear_group)
        disappear_group.add_to(mymap)

# --- Plot reappearances ---
if 'reappear_df' in globals() and not reappear_df.empty:
    for route, group_df in reappear_df.groupby("route_short_name"):
        reappear_group = FeatureGroup(name=f"{route} – Reappearances", show=False)
        for _, row in group_df.iterrows():
            if pd.notna(row["latitude"]) and pd.notna(row["longitude"]):
                Marker(
                    location=[row["latitude"], row["longitude"]],
                    icon=Icon(color="green", icon="check-circle", prefix="fa"),
                    tooltip=f"Vehicle {row.get('vehicle_id')} reappeared<br>{row.get('timestamp_collected')}"
                ).add_to(reappear_group)
        reappear_group.add_to(mymap)

# --- Plot route shapes ---
if 'shapes_df' in globals() and not shapes_df.empty:
    for route, group in shapes_df.groupby("route_short_name"):
        route_group = FeatureGroup(name=f"{route} – Route Shape", show=False)
        for shape_id, shape_data in group.groupby("shape_id"):
            shape_data = shape_data.sort_values("shape_pt_sequence")
            latlons = list(zip(shape_data["shape_pt_lat"], shape_data["shape_pt_lon"]))
            PolyLine(
                locations=latlons,
                color=color_map.get(route, "gray"),
                weight=2,
                opacity=0.6,
                popup=f"Route {route} | Shape {shape_id}"
            ).add_to(route_group)
        route_group.add_to(mymap)

# --- Add layer controls ---
LayerControl(collapsed=False).add_to(mymap)

# --- Display the map ---
mymap